In [25]:
# IRQIV submodule 1
###############################################################################
# 
# FLIRATSFiles.jl in /home/seth/.julia/dev/IRQIV/src/
# 
# Created: 2023-04-24 17:01:10 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-28 14:24:10 seth>
# 
###############################################################################

###############################################################################
module FLIRATSFiles
###############################################################################

###############################################################################
using Dates, Statistics, FileIO
using DocStringExtensions
import Base: size, length, getindex, lastindex # imported explicitely so that we can then overload with size(ats::ATSSequence)
###############################################################################

###############################################################################
export LoadATSImageSequence, isnuc, ats_ts
###############################################################################

###############################################################################
LoadATSImageSequence(FLIR_filename, I1inds, ats_year) = 
    ATSSequence(FLIR_filename, I1inds, ats_year)
###############################################################################



###############################################################################
"""
    ATSMetadata

composite type containing the minimal useful metadata of an ATS file for loading
images and BHP headers.
- Fields:
$(TYPEDFIELDS)
"""
struct ATSFileMetadata
    Name::String
    Path::String
    FilenameWithPath::String
    ATSnumFromName::String
    IntegrationTime::Float64
    FirstFrameCounter::Int
    Im1Offset::Int           # bytes
    ImHeight::Int            # pixels
    ImWidth::Int             # pixels
    BHPHeaderSize::Int       # bytes
    ImSize::Int              # bytes
    EstimatedTotalFrames::Int
    Estimatedfps::Float64
    FirstFrameTimestamp::DateTime
    FirstFrameTimestampString::String
end
# Outer constructors:
function ATSFileMetadata(metadata::ATSFileMetadata, ATSfps)
    """ Add/modify fps in metadata """
    return ATSFileMetadata(
        metadata.Name,
        metadata.Path,
        metadata.FilenameWithPath,
        metadata.ATSnumFromName,
        metadata.IntegrationTime,
        metadata.FirstFrameCounter,
        metadata.Im1Offset,
        metadata.ImHeight,
        metadata.ImWidth,
        metadata.BHPHeaderSize,
        metadata.ImSize,
        metadata.EstimatedTotalFrames,
        ATSfps,
        metadata.FirstFrameTimestamp,
    )
end
""" $(METHODLIST) """
function ATSFileMetadata(FilenameWithPath::String, IntegrationTime,
                         FirstFrameCounter, Im1Offset, ImHeight, ImWidth,
                         BHPHeaderSize, ImSize)
    Name = basename(FilenameWithPath)
    Path = dirname(FilenameWithPath)
    EstimatedTotalFrames = round(Int,
                                 (stat(FilenameWithPath).size-Im1Offset) /
                                     (BHPHeaderSize+ImSize))
    # Find ATS number from filename:
    rmatch = match(r"[0-9]{6}", FilenameWithPath)
    ATSnum = isa(rmatch, Void) ? "000000" : rmatch.match
    #
    ATSfps = NaN
    return ATSFileMetadata(Name, Path, FilenameWithPath, ATSnum, IntegrationTime,
                           FirstFrameCounter, Im1Offset, ImHeight, ImWidth,
                           BHPHeaderSize, ImSize, EstimatedTotalFrames, ATSfps, DateTime(0))
end
function ATSFileMetadata(FilenameWithPath::String, IntegrationTime::Float64,
                         FirstFrameCounter, Im1Offset, ImHeight, ImWidth,
                         BHPHeaderSize, ImSize, ATSfps, FirstFrameTimestamp)
    Name = basename(FilenameWithPath)
    Path = dirname(FilenameWithPath)
    EstimatedTotalFrames = round(Int,
                                 (stat(FilenameWithPath).size-Im1Offset) /
                                     (BHPHeaderSize+ImSize))
    # # This block returns the first match (sometimes this is something in the path):
    # Find ATS number from filename:
    # rmatch = match(r"[0-9]{6}", FilenameWithPath)
    # ATSnum = isa(rmatch, Void) ? "000000" : rmatch.match
    #
    # # This block returns the last match:
    Eachmatch = eachmatch(r"0[0-9]{5}", FilenameWithPath) # find all the regex matches
    # Eachmatch = eachmatch(r"[0-9]{6}", FilenameWithPath) # find all the regex matches
    Matches = map(x->getfield(x,:match), Eachmatch)      # extract all the patterns that were found
    ATSnum = isempty(Matches) ? "000000" : Matches[end]  # keep the last one or return 000000
    # 
    return ATSFileMetadata(Name, Path, FilenameWithPath, ATSnum, IntegrationTime,
                           FirstFrameCounter, Im1Offset, ImHeight, ImWidth,
                           BHPHeaderSize, ImSize, EstimatedTotalFrames, ATSfps, FirstFrameTimestamp)
end
function ATSFileMetadata(Name::String, Path::String, FilenameWithPath::String, ATSnumFromName,
                         IntegrationTime, FirstFrameCounter, Im1Offset,
                         ImHeight, ImWidth, BHPHeaderSize, ImSize,
                         EstimatedTotalFrames, Estimatedfps, FirstFrameTimestamp)
    return ATSFileMetadata(Name, Path, FilenameWithPath, ATSnumFromName,
               IntegrationTime, FirstFrameCounter, Im1Offset,
               ImHeight, ImWidth, BHPHeaderSize, ImSize,
               EstimatedTotalFrames, Estimatedfps, FirstFrameTimestamp, Dates.format(FirstFrameTimestamp, "yyyy-mm-dd HH:MM:SS"))
end
# 
function ATSFileMetadata(Name::String, Path::String,
                         FilenameWithPath::String, ATSnumFromName, IntegrationTime, FirstFrameCounter,
                         Im1Offset, ImHeight, ImWidth, BHPHeaderSize, ImSize, EstimatedTotalFrames,
                         Estimatedfps)
    return ATSFileMetadata(Name, Path, FilenameWithPath, ATSnumFromName,
               IntegrationTime, FirstFrameCounter, Im1Offset,
               ImHeight, ImWidth, BHPHeaderSize, ImSize,
               EstimatedTotalFrames, Estimatedfps, DateTime(0))
end
# 
function ATSFileMetadata(FilenameWithPath::String, IntegrationTime::Float64,
                         FirstFrameCounter, Im1Offset, ImHeight, ImWidth,
                         BHPHeaderSize, ImSize, ATSfps)
    return ATSFileMetadata(FilenameWithPath, IntegrationTime,
               FirstFrameCounter, Im1Offset, ImHeight, ImWidth,
               BHPHeaderSize, ImSize, ATSfps, DateTime(0))
end
###############################################################################
###############################################################################

###############################################################################
struct ATSSequence
    Frames::Array{UInt16,3}
    Headers::Array{UInt16,2}
    Metadata::ATSFileMetadata
    ATSSequence() = new()       # allow initialization of empty struct
    ATSSequence(Frames::Array{UInt16,3}, Headers::Array{UInt16,2}, Metadata::ATSFileMetadata) =
        new(Frames, Headers, Metadata)
end
# Outer constructors:
# ATSSequence(Frames::Array, Headers::Array, Metadata::ATSFileMetadata) = ATSSequence(Frames, Headers, Metadata)
""" $(DocStringExtensions.TYPEDSIGNATURES) """
function ATSSequence(Filename::String, frames=Inf, ats_year="auto")
    if ats_year=="auto"
        ats_year = stat(Filename).ctime |> # guess from file ctime
            Dates.unix2datetime |> _ctime -> Dates.format(_ctime, "yyyy") |> yr -> parse(Int,yr)
    end
    Metadata = FindATSFileMetadata(Filename; ATSfps=NaN, ats_year=ats_year)
    # Frames, BHPHeaders =
    Frames, Headers =
        if Base.length(frames) == 1 && isinf(frames); LoadATS(Filename);
        else
            if Base.length(frames) == 1; frames = 1:frames; end
            LoadMultipleFrames(Metadata, filter(x-> x <= Metadata.EstimatedTotalFrames, frames))
        end
    return ATSSequence(Frames, Headers,
                       ATSFileMetadata(Metadata, estimate_ats_fps(Headers))
                       )
end
###############################################################################
# ats_ts(ats::ATSSequence) = ParseBHPHeaderTimestamp(ats.BHPHeaders; ats_year=Dates.year(ats.ATSMetadata.FirstFrameTimestamp))
ats_ts(ats::ATSSequence) = ParseBHPHeaderTimestamp(ats.Headers; ats_year=Dates.year(ats.Metadata.FirstFrameTimestamp))
function ats_ts(ats::ATSSequence, frm::Union{Int, StepRange, Vector})
    # ParseBHPHeaderTimestamp(ats.BHPHeaders[frm,:]; ats_year=Dates.year(ats.ATSMetadata.FirstFrameTimestamp))
    ParseBHPHeaderTimestamp(ats.Headers[frm,:]; ats_year=Dates.year(ats.Metadata.FirstFrameTimestamp))
end
function ats_ts(ats::ATSSequence, frm::Symbol)
    if isequal(frm, :end)
        # return ats_ts(ats::ATSSequence, ats.ATSMetadata.EstimatedTotalFrames)
        return ats_ts(ats::ATSSequence, ats.Metadata.EstimatedTotalFrames)
    else
        @error("bad input!")
    end
end
# function ats_ts(ats::ATSSequence, frm::Union{Int, StepRange, Vector})
###############################################################################
# Overload some functions, to make it more convenient to access frame content:
size(ats::ATSSequence, args...) = Base.size(ats.Frames, args...)
length(ats::ATSSequence) = Base.size(ats.Frames,3)                       # return number of frames in sequence
getindex(ats::ATSSequence, args...) = Base.getindex(ats.Frames, args...) # e.g., ats[:,:,3]
getindex(ats::ATSSequence, imnumber::Integer) = Base.getindex(ats.Frames, :,:,imnumber) # e.g., ats[3] returns the 3rd frame in the sequence
lastindex(ats::ATSSequence) = Base.size(ats.Frames, 3) # 
###############################################################################


###############################################################################
function FindATSFileMetadata(ATSFilenameWithPath::String; ATSfps=NaN, ats_year="auto")
    FileMetadata = open(ATSFilenameWithPath, "r") do ATSfileIOStream
        FindATSFileMetadata(ATSfileIOStream; ATSfps=ATSfps, ats_year=ats_year)
    end
end
# 
function FindATSFileMetadata(ATSfileIOStream::IOStream; ATSfps=NaN, ats_year="auto")
    PixelsBlockHeader = Dict()
    ATS_PixelsBlock_GUID              = "48A16EF9-457E-47E8-9F70-5495781A1B64"
    # Read file only until the first pixels block, to get the minimal metadata we need:
    IOStreamIncomingPosition =  Base.mark(ATSfileIOStream) # mark the current position within the stream - we'll go back here at the end
    # 
    Base.seekstart(ATSfileIOStream)
    while !eof(ATSfileIOStream)              # i.e., while we have not yet read the end-of-file character
        BlockHeader = ReadBlockHeader(ATSfileIOStream)
        if BlockHeader["GUID"] == ATS_PixelsBlock_GUID
            PixelsBlockHeader["StructSize"]                = Int(read(ATSfileIOStream, UInt32)) # Size in bytes of the pixels block structure (512).
            PixelsBlockHeader["Version"]                   = Int(read(ATSfileIOStream, UInt32))
            PixelsBlockHeader["TotalSize"]                 = Int(read(ATSfileIOStream, UInt32)) # Size in bytes of the pixels block structure plus the frame block configuration data size (TotalFrameBlockConfigSize)
            PixelsBlockHeader["PixelType"]                 = ReadGUID(ATSfileIOStream)
            PixelsBlockHeader["PixelWidth"]                = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["ImageWidth"]                = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["ImageHeight"]               = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["FrameWidth"]                = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["FrameHeight"]               = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["PackedWidth"]               = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["PackedHeight"]              = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["NumFramesPerPackedFrame"]   = Int(read(ATSfileIOStream, UInt16))
            PixelsBlockHeader["PackedSize"]                = Int(read(ATSfileIOStream, UInt32))
            PixelsBlockHeader["TotalFrameSize"]            = Int(read(ATSfileIOStream, UInt32))
            PixelsBlockHeader["TotalFrameBlockSize"]       = Int(read(ATSfileIOStream, UInt32))
            PixelsBlockHeader["TotalFrameBlockConfigSize"] = Int(read(ATSfileIOStream, UInt32))
            # PixelsBlockHeader["padding"]                   = read(ATSfileIOStream, UInt8, 68)
            PixelsBlockHeader["padding"]                   = read!(ATSfileIOStream, Array{UInt8}(undef, 68))
            for Frameblk_n = 1:8
                Frameblk = "FrameBlock_$Frameblk_n"
                PixelsBlockHeader[Frameblk] = Dict()
                PixelsBlockHeader[Frameblk]["GUID"]                 = ReadGUID(ATSfileIOStream)
                PixelsBlockHeader[Frameblk]["FrameBlockSize"]       = read(ATSfileIOStream, UInt32)
                PixelsBlockHeader[Frameblk]["FrameBlockConfigSize"] = read(ATSfileIOStream, UInt32)
                # PixelsBlockHeader[Frameblk]["padding"]              = read(ATSfileIOStream, UInt8, 8)
                PixelsBlockHeader[Frameblk]["padding"]              = read!(ATSfileIOStream, Array{UInt8}(undef, 8))
            end
            # PixelsBlockHeader["padding2"] = read(ATSfileIOStream, UInt8, 128)
            PixelsBlockHeader["padding2"] = read!(ATSfileIOStream, Array{UInt8}(undef, 128))
            break
        else
            seek(ATSfileIOStream, BlockHeader["EndPosition"])
        end
    end
    # 
    ImageHeight = PixelsBlockHeader["ImageHeight"]
    ImageWidth = PixelsBlockHeader["ImageWidth"]
    FrameWidth = PixelsBlockHeader["FrameWidth"]
    BHPHeaderSize_bytes = PixelsBlockHeader["FrameWidth"]*2 # This is really just the number of bytes in the first row of 16-bit pixels
    ImageSize_bytes = PixelsBlockHeader["TotalFrameSize"]-BHPHeaderSize_bytes
    #
    Image1Offset = position(ATSfileIOStream)
    #
    # read the first BHPHeader:
    # Need to add something to read a few more BHPHeaders and calculate a Δt from them
    seek(ATSfileIOStream, Image1Offset)
    # BHPHeader = read(ATSfileIOStream, UInt16, ImageWidth)
    BHPHeader = read!(ATSfileIOStream, Array{UInt16,1}(undef, ImageWidth))
    IntegrationTime, _, FrameCounter, _, _, _, _ = ParseKeyMetadatFromBHPHeader(BHPHeader)
    # 
    FirstFrameTimestamp = ParseBHPHeaderTimestamp(BHPHeader; ats_year=ats_year)
    #
    ATSFilenameWithPath = convert(String, split(ATSfileIOStream.name, r"<file |>"; keepempty=false)[]) # return the file name
    # ATSFilenameWithPath = split(ATSfileIOStream.name, r"<file |>"; keepempty=false)[] # return the file name
    FileMetadata = ATSFileMetadata(ATSFilenameWithPath,
                                   IntegrationTime[1], # convert array to Float
                                   FrameCounter[1],    # convert array to Int
                                   Image1Offset,
                                   ImageHeight,
                                   ImageWidth,
                                   BHPHeaderSize_bytes,
                                   ImageSize_bytes,
                                   ATSfps,
                                   FirstFrameTimestamp,
                                   )
    if isnan(ATSfps)
        ATSfps = estimate_ats_fps(FileMetadata)
        FileMetadata = ATSFileMetadata(
            FileMetadata.Name,
            FileMetadata.Path,
            FileMetadata.FilenameWithPath,
            FileMetadata.ATSnumFromName,
            FileMetadata.IntegrationTime,
            FileMetadata.FirstFrameCounter,
            FileMetadata.Im1Offset,
            FileMetadata.ImHeight,
            FileMetadata.ImWidth,
            FileMetadata.BHPHeaderSize,
            FileMetadata.ImSize,
            FileMetadata.EstimatedTotalFrames,
            ATSfps,             # Estimatedfps::Float64
            FileMetadata.FirstFrameTimestamp)
    end    
    Base.seek(ATSfileIOStream, IOStreamIncomingPosition) # go back to where we were at the start
    return FileMetadata
end
###############################################################################

###############################################################################
function LoadSingleFrame(FilenameWithPath::String, FrameNum=1; kwargs...)
    FileMetadata = FindATSFileMetadata(FilenameWithPath)
    return LoadSingleFrame(FileMetadata, FrameNum; kwargs...)
end
function LoadSingleFrame(FileMetadata::ATSFileMetadata, FrameNum=1; kwargs...)
    Frame, BHPHeader = open(FileMetadata.FilenameWithPath, "r") do f
        Frame, BHPHeader = LoadSingleFrame(f, FileMetadata, FrameNum; kwargs...)
    end
    return UInt16.(Frame), UInt16.(BHPHeader)
end
function LoadSingleFrame(f::IOStream, FileMetadata::ATSFileMetadata, FrameNum;
                         ReadFrame = true,
                         ReadHeader = true,
                         kwargs...)
    seek(f, FileMetadata.Im1Offset + (FileMetadata.BHPHeaderSize+FileMetadata.ImSize)*(FrameNum-1))
    # skip the parts we weren't asked for, to improve performance:
    if ReadHeader
        # BHPHeader = read(f, UInt16, FileMetadata.ImWidth) # read and store the header line
        BHPHeader = read!(f, Array{UInt16}(undef, FileMetadata.ImWidth)) # read and store the header line
    else
        BHPHeader = []
        skip(f, FileMetadata.BHPHeaderSize)
    end
    if ReadFrame
        # Frame = reshape(read(f, UInt16, FileMetadata.ImWidth*FileMetadata.ImHeight),
        Frame = reshape(read!(f, Array{UInt16}(undef, FileMetadata.ImWidth*FileMetadata.ImHeight)),
                        FileMetadata.ImWidth, FileMetadata.ImHeight)'
    else
        Frame = []
        skip(f, FileMetadata.ImSize)
    end
    return UInt16.(Frame), UInt16.(BHPHeader)
end
#
function LoadSingleFrame!(Frame, BHPHeader, f::IOStream, FileMetadata::ATSFileMetadata, FrameNum)
    seek(f, FileMetadata.Im1Offset + (FileMetadata.BHPHeaderSize+FileMetadata.ImSize)*(FrameNum-1))
    BHPHeader .= read!(f, Array{UInt16}(undef, FileMetadata.ImWidth)) # read and store the header line
    Frame .= reshape(read!(f, Array{UInt16}(undef, FileMetadata.ImWidth*FileMetadata.ImHeight)),
                               FileMetadata.ImWidth, FileMetadata.ImHeight)'
end
###############################################################################

###############################################################################
function LoadMultipleFrames(FilenameWithPath::String, Nframes::Int; kwargs...)
    FileMetadata = FindATSFileMetadata(FilenameWithPath)
    FrameNums = unique(round.(Int,range(1,stop=FileMetadata.EstimatedTotalFrames,length=Nframes)))
    return LoadMultipleFrames(FilenameWithPath, FrameNums; kwargs...)
end
function LoadMultipleFrames(FilenameWithPath::String, FrameNums::AbstractRange; kwargs...)
    return LoadMultipleFrames(FilenameWithPath, collect(FrameNums))
end
function LoadMultipleFrames(FilenameWithPath::String, FrameNums::Array; # changed 2022-09-23
                            ReadFrame = true,
                            ReadHeader = true,
                            kwargs...)
    FileMetadata = FindATSFileMetadata(FilenameWithPath)
    return LoadMultipleFrames(FileMetadata, collect(FrameNums))
end
function LoadMultipleFrames(ATSFileMetadata::ATSFileMetadata, FrameNums::Union{Vector,Array};
                            ReadFrame = true,
                            ReadHeader = true,
                            kwargs...)
    if any(isinf.(FrameNums))
        FrameNums[isinf.(FrameNums)] = [ATSFileMetadata.EstimatedTotalFrames]
        FrameNums = Int.(FrameNums)
    end
    FrameNums = FrameNums[ FrameNums .<= ATSFileMetadata.EstimatedTotalFrames ]
    if ReadFrame & ReadHeader
        Frames = Array{UInt16,3}(undef, ATSFileMetadata.ImHeight, ATSFileMetadata.ImWidth, length(FrameNums))
        BHPHeaders= Array{UInt16,2}(undef, length(FrameNums), ATSFileMetadata.ImWidth)
        open(ATSFileMetadata.FilenameWithPath, "r") do f
            for i in 1:length(FrameNums)
                Frames[:,:,i], BHPHeaders[i,:] = LoadSingleFrame(f, ATSFileMetadata, FrameNums[i]; ReadFrame=ReadFrame, ReadHeader=ReadHeader, kwargs...)
            end
        end
    elseif ReadFrame & !ReadHeader
        Frames = Array{UInt16,3}(undef, ATSFileMetadata.ImHeight, ATSFileMetadata.ImWidth, length(FrameNums))
        BHPHeaders = []
        open(ATSFileMetadata.FilenameWithPath, "r") do f
            for i in 1:length(FrameNums)
                Frames[:,:,i], _ = LoadSingleFrame(f, ATSFileMetadata, FrameNums[i]; ReadFrame=ReadFrame, ReadHeader=ReadHeader, kwargs...)
            end
        end
    elseif ReadHeader & !ReadFrame
        Frames = []
        BHPHeaders= Array{UInt16,2}(undef, length(FrameNums), ATSFileMetadata.ImWidth)
        open(ATSFileMetadata.FilenameWithPath, "r") do f
            for i in 1:length(FrameNums)
                _, BHPHeaders[i,:] = LoadSingleFrame(f, ATSFileMetadata, FrameNums[i]; ReadFrame=ReadFrame, ReadHeader=ReadHeader, kwargs...)
            end
        end
    else
        error("invalid options provided! kwargs: $kwargs")
    end
    return Frames, BHPHeaders
end
###############################################################################


###############################################################################
function estimate_ats_fps(BHPHeaders::Array{UInt16,2})
    # size(BHPHeaders,1) < 3 ? error("Too few BHPHeaders!") : nothing
    if size(BHPHeaders,1) < 3 
        @warn("Too few BHPHeaders to calculate fps!")
        return NaN
    else
        ts = ParseBHPHeaderTimestamp(BHPHeaders)
        # ATSfps = 1000 / convert(Float64, Dates.value(mean(diff(ts))))
        ATSfps = 1000 / convert(Float64, mean(Dates.value.(diff(ts))))
        return ATSfps
    end
end
function estimate_ats_fps(ATSFilenameWithPath::String)
    _, BHPHeaders = LoadMultipleFrames(ATSFilenameWithPath, 1:5)
    return estimate_ats_fps(BHPHeaders)
end
function estimate_ats_fps(ATSFileMetadata::ATSFileMetadata)
    _, BHPHeaders = LoadMultipleFrames(ATSFileMetadata, collect(1:5))
    return estimate_ats_fps(BHPHeaders)
end
###############################################################################

###############################################################################
function BlockTypeFromGUID(GUID::String)
    # ----------
    ATS_IDBlock_GUID                  = "9C3C13F6-1A10-49C5-86C9-ACF8A928DCC6"
    ATS_SourceInfoBlock_GUID          = "344ED254-76DC-4B4C-B47B-3548A6837C26"
    ATS_AbateCfgBlock_GUID            = "12C99632-E051-4922-B9A3-A7259271C94D"
    ATS_ObjectParametersBlock_GUID  = "616033EC-ED49-485A-B1FE-4F563DC2F52" # Use this in Julia
    # ATS_ObjectParametersBlock_GUID    = "616033EC-ED49-485A-B1FE-4F563DC20F52" # Use this in Matlab
    ATS_NicevilleFactoryCalBlock_GUID = "7ED232C7-2ECF-4552-BB67-D406D5E87E61"
    ATS_PixelsBlock_GUID              = "48A16EF9-457E-47E8-9F70-5495781A1B64"
    ATS_BHP_SC_pixel_type_GUID        = "762A7AEB-B98E-4628-8A7B-7AEE737F7754"
    # ----------
    if GUID ==  "9C3C13F6-1A10-49C5-86C9-ACF8A928DCC6"
        BlockType = "ATS_IDBlock"
    elseif GUID ==  "344ED254-76DC-4B4C-B47B-3548A6837C26"
        BlockType = "ATS_SourceInfoBlock"
    elseif GUID ==  "12C99632-E051-4922-B9A3-A7259271C94D"
        BlockType = "ATS_AbateCfgBlock"
    elseif GUID ==  "616033EC-ED49-485A-B1FE-4F563DC2F52" # Use this in Julia
        BlockType = "ATS_ObjectParametersBlock"
        # ATS_ObjectParametersBlock_GUID    = "616033EC-ED49-485A-B1FE-4F563DC20F52" # Use this in Matlab
    elseif GUID ==  "7ED232C7-2ECF-4552-BB67-D406D5E87E61"
        BlockType = "ATS_NicevilleFactoryCalBlock"
    elseif GUID ==  "48A16EF9-457E-47E8-9F70-5495781A1B64"
        BlockType = "ATS_PixelsBlock"
    elseif GUID ==  "762A7AEB-B98E-4628-8A7B-7AEE737F7754"
        BlockType = "ATS_BHP_SC_pixel_type"
    elseif GUID ==  "00000000-0000-0000-0000-000000000000"
        BlockType = "ATS_Free_Block"
    else
        BlockType = []
    end
    return BlockType
end
###############################################################################

###############################################################################
function ReadGUID(f::IOStream)
    GUID1 = string(read(f, UInt32), base=16, pad=8) # 8 chars
    GUID2 = join(string.(read!(f, Array{UInt16,1}(undef, 2)), base=16, pad=4), "-") # 4*2 chars
    GUID3 = join(string.(read!(f, Array{UInt8,1}(undef, 2)), base=16, pad=2)) # 2*2 chars
    GUID4 = join(string.(read!(f, Array{UInt8,1}(undef, 6)), base=16, pad=2)) # 6*2 chars (2 chars in each of 6 strings)
    GUID = uppercase(join([GUID1, GUID2, GUID3, GUID4], "-"))
    return GUID
end
###############################################################################

###############################################################################
function ReadBlockHeader(f)
    BlockHeader                         = Dict()
    BlockHeader["HeaderStartPosition"]  = position(f)
    BlockHeader["Name"]                 = String([read(f, Char) for i=1:32])
    BlockHeader["GUID"]                 = ReadGUID(f)
    BlockHeader["Version"]              = read(f, UInt32)
    BlockHeader["Preset"]               = read(f, UInt8) # Preset this block belongs to (0 – 3), -1 if not per preset.
    BlockHeader["Frame"]                = read(f, UInt64) # Frame number this block belongs to, UInt64Max if it doesn’t belong to a particular frame
    BlockHeader["Flags"]                = read(f, UInt32) # Flags 0x1 = block valid, 0x2 = CRC valid.
    BlockHeader["Size"]                 = read(f, UInt64) # Size of the payload
    BlockHeader["CRC"]                  = read(f, UInt32)  # CRC of the payload (0 if CRC is not valid)
    BlockHeader["PayloadStartPosition"] = position(f)
    if BlockHeader["Size"] == typemax(UInt64)
        mark(f)                 # mark the present position
        BlockHeader["EndPosition"]          = position(seekend(f)) # find the end of the file
        reset(f)                                                   # return to the previous position and remove the mark
    else
        BlockHeader["EndPosition"]          = position(f) + Int(BlockHeader["Size"])
    end
    # BlockHeader["EndPosition"]          = position(f) + BlockHeader["Size"]
    BlockHeader["BlockType"]            = BlockTypeFromGUID(BlockHeader["GUID"])
    return BlockHeader
end
###############################################################################

###############################################################################
function ParseKeyMetadatFromBHPHeader(BHPHeadersStack)
    Base.size(BHPHeadersStack,2) == 1 ? BHPHeadersStack = reshape(BHPHeadersStack, 1, Base.length(BHPHeadersStack)) : nothing
    # # Status Register is the 6th pixel
    # Bit 5 of this register is whether the NUC flag is deployed or not
    # NUCFlagState = [parse(Bool,bin(BHPHeadersStack[i,6],16)[11]) for i in 1:size(BHPHeadersStack,1)] # Extract only the relevant bit.
    NUCFlagState = [parse(Bool,string(BHPHeadersStack[i,6],base=2,pad=16)[11]) for i in 1:size(BHPHeadersStack,1)] # Extract only the relevant bit.
    FPorChassisTemp = convert(Array{Int},BHPHeadersStack[:,7])/100 .- 273.15 # raw value is K*100
    AirGapTemp = convert(Array{Int},BHPHeadersStack[:,8])/100 .- 273.15      # raw value is K*100
    DDCATemp = convert(Array{Int},BHPHeadersStack[:,9])/100 .- 273.15        # raw value is K*100
    FPATemp = convert(Array{Int},BHPHeadersStack[:,10])/100 .- 273.15        # raw value is K*100
    FrameCounter = UInt16ToInt32(BHPHeadersStack[:,12], BHPHeadersStack[:,11])
    IntegrationTime = UInt16ToInt32(BHPHeadersStack[:,14], BHPHeadersStack[:,13]) * 160e-9 # Each count in the (integer) raw value represents a 160ns period.
    # return IntegrationTime, NUCFlagState, FrameCounter, FPorChassisTemp, AirGapTemp, DDCATemp, FPATemp
    return (IntegrationTime=IntegrationTime, NUCFlagState=NUCFlagState, FrameCounter=FrameCounter,
            FPorChassisTemp=FPorChassisTemp, AirGapTemp=AirGapTemp, DDCATemp=DDCATemp, FPATemp=FPATemp)
end
###############################################################################


###############################################################################
"""
        UInt16ToInt32(Upper::UInt16,Lower::UInt16)
        UInt16ToInt32(Upper::Array{UInt16},Lower::Array{UInt16})

    Takes two UInt16 partial words and combines them into a single Int32.

    (note that this function can probably be reimplemented using the built in reintepret function)

    """
function UInt16ToInt32(Upper::UInt16,Lower::UInt16)
    # parse(Int32,"$(bin(Upper,16))$(bin(Lower,16))",2)
    parse(Int32,"$(string(Upper,base=2,pad=16))$(string(Lower,base=2,pad=16))",base=2)
end
function UInt16ToInt32(Upper::Array{UInt16},Lower::Array{UInt16})
    # 
    [UInt16ToInt32(Upper[i],Lower[i]) for i in 1:Base.length(Upper)]
end
###############################################################################

###############################################################################
# function ParseBHPHeaderTimestamp(BHPHeadersStack; ats_year=2014)
#     return ParseBHPHeaderTimestamp([BHPHeadersStack]; ats_year=ats_year)
# end
# -----------------------------------------------------------------------------
"""
    ParseBHPHeaderTimestamp(BHPHeadersStack::Array{UInt16,1}; ats_year="auto")

Calculates a timestamp from an ats file BHP Header.

Selects year from a lookup table based on FrameCounter (the camera's ordinal
count of all frames recorded).

Applies a timestamp correction for the camera's clock not being correctly set
during the first part of the November 2017 experiments at Sutter Slough and
Isleton Bridge (as identified by the  FrameCounter value).
"""
function ParseBHPHeaderTimestamp(BHPHeadersStack::Array{UInt16,1}; ats_year=2017)
    ## From BHP header documentation of the structure of the IRIG word:
    # Bits 5-0: Second
    # Bits 11-6: Minute
    # Bits 17-12: Hour
    # Bits 26-18: Day
    # Bit 31: Locked
    # 
    ## FLIR's bit 0 is the LSB, which is bit 32 (or 'end') when referencing as
    ## a julia array.
    # 
    # Explanation of the date conversion code block: for each timestamp (irig),
    # convert from an Int32 to a string of the binary bits (this is
    # bits(irig)). Then select only the relevant bits for the time period of
    # interest (days, hours, etc.), and parse this string back to an interger
    # (parse() ,which takes as arguments the output type, string, and base to
    # convert from ).
    # 
    IRIGword = UInt16ToInt32(BHPHeadersStack[2], BHPHeadersStack[3])
    IRIGMicrosecond= UInt16ToInt32(BHPHeadersStack[4], BHPHeadersStack[5])
    IRIGSecond     = parse(Int,bitstring(IRIGword)[end-5:end],base=2)
    IRIGMinute     = parse(Int,bitstring(IRIGword)[end-11:end-6],base=2)
    IRIGHour       = parse(Int,bitstring(IRIGword)[end-17:end-12],base=2)
    IRIGDay        = parse(Int,bitstring(IRIGword)[end-26:end-18],base=2)
    IRIGRemainder  = parse(Int,bitstring(IRIGword)[end-31:end-27],base=2)
    IRIGRemainder != 0 ? error("Problem with IRIG date conversion!") : nothing
    # This was added in October 2018 to correct for the camera's clock not being
    # synced before the start of the November 2017 expreriments. It could be
    # modified to automatically select the year from the frame counter!
    FrameCounter = UInt16ToInt32(BHPHeadersStack[12], BHPHeadersStack[11])
    if isa(ats_year, String) && occursin(r"auto"i, ats_year)
        if  minimum(FrameCounter) <  13623
            ats_year = 2014
        elseif 13623 <= minimum(FrameCounter) < 19392 # this is equivalent to (2904 <= ATSnum < 3028)
            ats_year = 2017
        elseif 19392 <= minimum(FrameCounter) # this is equivalent to 
            ats_year = 2019
        elseif occursin(r"mtime"i, ats_year)
            ats_year = Dates.year(unix2datetime(mtime(ATSFilenameWithPath)))
        end
    end
    # the following was corrected on 2018-01-10 to remove the erroneous offset of one day:
    Timestamp = DateTime(ats_year) +       # this returns Jan 1 in year "ats_year" - introduces an offset of one day!
    Dates.Day(IRIGDay-1) + # subtract one day to start from time 0 in the year + the number of days we actually want
        Dates.Hour(IRIGHour) +
        Dates.Minute(IRIGMinute) +
        Dates.Second(IRIGSecond) +
        Dates.Millisecond(round(IRIGMicrosecond/1000))
    # Added in October 2018: correct ~61 minute offset in camera's clock time
    # due to DST and clock drift after not being sync'd for 3 years (Spring 2014
    # to Fall 2017).
    if 13623 <= FrameCounter <  401433 # this is equivalent to (2904 <= ATSnum < 3028)
        Timestamp += Dates.Millisecond(-3666849)
    end
    return Timestamp
end
function ParseBHPHeaderTimestamp(BHPHeadersStack::Array{UInt16,2}; ats_year="auto")
    # function ParseBHPHeaderTimestamp(BHPHeadersStack::Array{UInt16,2}; ats_year=2014)
    Timestamps = Array{DateTime,1}(undef,size(BHPHeadersStack,1))
    for row in 1:size(BHPHeadersStack,1)
        Timestamps[row] = ParseBHPHeaderTimestamp(BHPHeadersStack[row,:]; ats_year=ats_year)
    end
    return Timestamps
end


###############################################################################
function ParseBHPHeaderNUCStatus(BHPHeadersStack::Array{UInt16,2})
    NUCFlagState = [ parse(Bool, x[11]) for x in bitstring.(BHPHeadersStack[:,6]) ]
end
function ParseBHPHeaderNUCStatus(BHPHeadersStack::Array{UInt16,1})
    NUCFlagState = parse(Bool,bitstring(BHPHeadersStack[6])[11]) # Extract only the relevant bit.
end
isnuc = ParseBHPHeaderNUCStatus
###############################################################################

###############################################################################
end                             # end of module
###############################################################################


Main.FLIRATSFiles

In [26]:
# IRQIV submodule 2
###############################################################################
# 
# CameraSpecifications.jl in /home/seth/.julia/dev/IRQIV/src/
# 
# Created: 2023-04-26 11:32:13 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-27 10:43:36 seth>
# 
###############################################################################

###############################################################################
module CameraSpecifications
###############################################################################

###############################################################################
# structs:
export
    IntrinsicCalibrationParameters,
    CameraSpecs
# Camera specs:
export
    FLIR_SC8303_17mm,
    FLIR_SC8303_50mm
###############################################################################

###############################################################################
struct CameraSpecs
    Manufacturer::String             # camera manufacturer
    Camera::String                   # camera model
    Lens::String                     # lens name / description
    pixel_pitch_m::Float64           # pixel spacing in FPA [m]
    pixel_resolution::Array{Int64,1} # n pixels in Horizontal,Vertical
    FOV_rad::Array{Float64,1}        # Horizontal,Vertical view angle [Radians]
    IFOV_rad::Array{Float64,1}       # Horizontal,Vertical single pixel view angle [Radians]
    nominal_focal_length_m::Float64  # [m]
    nominal_focal_length_px::Float64  # [pixels]
    NETD_mK::Float64                 # [mK]
    spectral_range_μm::Array{Float64,1} # [μm]
    nominal_ifov_mrad::Float64          # [mRad]
    calculated_ifov_rad_FOV_pixres::Array{Float64,1} # [Rad]
    calculated_ifov_rad_pixpitch_flength::Float64 # [Rad]
    f_number::Float64                   # nondim
    u₀::Float64
    v₀::Float64
    λᵤ::Float64
    λᵥ::Float64
    # Radial distortion coefficients:
    radial_distortion_poly_degrees::Array{Int64,1}  # degrees of those coefficients in the radial distortion polynomial
    radial_distortion_poly_coeffs::Array{Float64,1} # coefficients of the radial distortion polynomial
    # 
end
###############################################################################


###############################################################################
# If any of nominal_ifov, u₀, v₀, λᵤ, λᵥ, are not provided, estimate them.
function CameraSpecs(Manufacturer, Camera, Lens, pixel_pitch_m, pixel_resolution, FOV_rad,
                     nominal_focal_length_m, NETD_mK, spectral_range_μm,
                     nominal_ifov_mrad, f_number, u₀, v₀, λᵤ,
                     λᵥ, radial_distortion_poly_degrees, radial_distortion_poly_coeffs)
    calculated_ifov_rad_FOV_pixres = FOV_rad./pixel_resolution
    nominal_focal_length_px = nominal_focal_length_m / pixel_pitch_m
    calculated_ifov_rad_pixpitch_flength = pixel_pitch_m / nominal_focal_length_m
    if ~any([isnan.(nominal_ifov_mrad)..., isempty.(nominal_ifov_mrad)...])
        IFOV_rad = FOV_rad ./ pixel_resolution
    else
        if ~any([isnan.(calculated_ifov_rad_FOV_pixres)..., isempty.(calculated_ifov_rad_FOV_pixres)...])
            IFOV_rad = calculated_ifov_rad_FOV_pixres
        else
            IFOV_rad = calculated_ifov_rad_pixpitch_flength
        end
    end
    length(IFOV_rad) == 1 ? IFOV_rad = [IFOV_rad,IFOV_rad] : nothing
    isnan(u₀) ? u₀ = sum([1,pixel_resolution[1]])/2 : nothing
    isnan(v₀) ? v₀ = sum([1,pixel_resolution[2]])/2 : nothing
    isnan(λᵤ) ? λᵤ = 1.0 : nothing
    isnan(λᵥ) ? λᵥ = 1.0 : nothing
    # If no radial distortion information is provided, assume zero distortion:
    isempty(radial_distortion_poly_degrees) ? radial_distortion_poly_degrees = [1.0] : nothing
    isempty(radial_distortion_poly_coeffs) ? radial_distortion_poly_coeffs = [0] : nothing
    # 
    return CameraSpecs(
        Manufacturer, Camera, Lens, pixel_pitch_m, pixel_resolution, FOV_rad, IFOV_rad,
        nominal_focal_length_m, nominal_focal_length_px, NETD_mK,
        spectral_range_μm, nominal_ifov_mrad,
        calculated_ifov_rad_FOV_pixres,
        calculated_ifov_rad_pixpitch_flength, f_number,
        u₀, v₀, λᵤ, λᵥ,
        radial_distortion_poly_degrees, radial_distortion_poly_coeffs)
end
###############################################################################

###############################################################################
"""
    IntrinsicCalibrationParameters

If no input is provided, default values are assigned

Fields:

    u₀ :: Float64
    v₀ :: Float64
    λᵤ :: Float64
    λᵥ :: Float64
    PixelPitch :: Float64
    radial_distortion_poly_coeffs::Array{Float64}
    radial_distortion_poly_degrees::Array{Int}
"""
struct IntrinsicCalibrationParameters
    u₀::Float64
    v₀::Float64
    λᵤ::Float64
    λᵥ::Float64
    PixelPitch::Float64
    radial_distortion_poly_degrees::Array{Float64,1}
    radial_distortion_poly_coeffs::Array{Float64,1}
    # Or use trivial default values (if called with no inputs):
end
IntrinsicCalibrationParameters() =
    IntrinsicCalibrationParameters(0.0, # u₀, [pixels]                    
                                   0.0, # v₀, [pixels]                    
                                   1.0, # λᵤ, [dimensionless - ratio]     
                                   1.0, #  λᵥ, [dimensionless - ratio]    
                                   1.0, # , # PixelPitch, [meters/pixel]
                                   [1.0],     # radial_distortion_poly_degrees
                                   [1.0])   # radial_distortion_poly_coeffs
###############################################################################

###############################################################################
# Generate from CameraSpecs:
IntrinsicCalibrationParameters(Camera::CameraSpecs) =
    IntrinsicCalibrationParameters(Camera.u₀,
                                   Camera.v₀,
                                   Camera.λᵤ,
                                   Camera.λᵥ,
                                   Camera.pixel_pitch_m,
                                   Camera.radial_distortion_poly_degrees,
                                   Camera.radial_distortion_poly_coeffs)
###############################################################################

###############################################################################
FLIR_SC8303_17mm = CameraSpecs("FLIR", "SC8303",    "17mm", 14e-6 , [1344,  784], deg2rad.([57.9,35.7]), 17e-3, 25.0, [3.0,5.0], NaN, 4.0, NaN, NaN, 1.0086485610865155, 1.0, [1, 3, 5], [0.020522291524825344, -3.156396095419575e-8, -8.859823756718999e-14])
FLIR_SC8303_50mm = CameraSpecs("FLIR", "SC8303", "50mm", 14e-6 , [1344,784], deg2rad.([21.3,12.5]), 50e-3, 25.0, [3.0,5.0], NaN, 4.0, NaN, NaN, NaN, NaN, [], [])
###############################################################################

###############################################################################
end                             # end of module
###############################################################################


Main.CameraSpecifications

In [27]:
# IRQIV submodule 3
###############################################################################
# 
# ExtrinsicCameraCalibration.jl in /home/seth/Dropbox-sas262/Dropbox/Julia/MyLibraries/
# 
# Created: 2016-12-09 10:28:28 by seth@sas-mbp
# last changed Time-stamp: <2023-04-27 12:43:05 seth>
# 
###############################################################################


###############################################################################
"""
# ExtrinsicCameraCalibration
Functions and data structures related to extrinsic camera calibration. Generally follows Holland et al., 1997.

### References:
Holland, K., Holman, R., Lippmann, T., Stanley, J., & Plant, N. (1997). Practical use of video imagery in nearshore oceanographic field studies. IEEE JOURNAL OF OCEANIC ENGINEERING, 22(1), 81–92. http://dx.doi.org/https://doi.org/10.1109/48.557542 
"""
module ExtrinsicCameraCalibration
###############################################################################

###############################################################################
using Printf
using Statistics
import IRQIV: CameraSpecs, IntrinsicCalibrationParameters
###############################################################################

###############################################################################
export
    ExtrinsicCalibration, CalculateDLTcoefficients
# structs:
export
    ExtrinsicCalibrationParameters, DLT
###############################################################################

###############################################################################
"""
    ExtrinsicCalibrationParameters

Creates a DataType with the extrinsic calibration parameters.

If no input is provided, default values are assigned, found from calibrating
based on 7 of the 8 GCPs defined for ATS file 2050, with the excluded GCP being
the waterline of ADCP3's piling.
"""
struct ExtrinsicCalibrationParameters
    φ::Float64   # azimuth. clockwise from the north (y-axis).
    τ::Float64   # tilt. angle away from a vertical line pointing up from the camera. this will generally be negative, or alternatively >180 degrees
    σ::Float64   # roll. rotation of the camera around the centeral ray.
    f::Float64   # distance b/w FPA and optical center of camera, in pixels (i.e., meters/pixel pitch). Note that this may be a negative number (if the FPA is behind the focal point)
    x_c::Float64 # x coordinate of the camera's optical center (e.g., UTM East)
    y_c::Float64 # y coordinate of the camera's optical center (e.g., UTM North)
    z_c::Float64 # z coordinate of the camera's optical center (e.g., above vertical datum)
end
# Initialize with all nans (if called with no inputs):
ExtrinsicCalibrationParameters() = new(NaN, # φ,  azimuth [radians]        
                                       NaN, # τ,  tilt (pitch) [radians]   
                                       NaN, # σ,  roll [radians]           
                                       # 
                                       NaN, # f, [pixels]                 
                                       # 
                                       NaN, # x_c, [meters]                 
                                       NaN, # y_c, [meters]                 
                                       NaN) # z_c, [meters]                 
###############################################################################

##############################################################################
struct RotationMatrix <: AbstractFloat
    m11::Float64
    m12::Float64
    m13::Float64
    m21::Float64
    m22::Float64
    m23::Float64
    m31::Float64
    m32::Float64
    m33::Float64
end
RotationMatrix(χ::ExtrinsicCalibrationParameters) = RotationMatrix(
    cos(χ.φ) * cos(χ.σ) + sin(χ.φ) * cos(χ.τ) * sin(χ.σ),  # m11
    -sin(χ.φ) * cos(χ.σ) + cos(χ.φ) * cos(χ.τ) * sin(χ.σ), # m12
    sin(χ.τ) * sin(χ.σ),                                   # m13
    #                                                           
    -cos(χ.φ) * sin(χ.σ) + sin(χ.φ) * cos(χ.τ) * cos(χ.σ), # m21
    sin(χ.φ) * sin(χ.σ) + cos(χ.φ) * cos(χ.τ) * cos(χ.σ),  # m22
    sin(χ.τ) * cos(χ.σ),                                   # m23
    #                                                           
    sin(χ.φ) * sin(χ.τ),                                   # m31
    cos(χ.φ) * sin(χ.τ),                                   # m32
    -cos(χ.τ),                                             # m33
)
##############################################################################


###############################################################################
struct DLT
    L₁::Float64
    L₂::Float64
    L₃::Float64
    L₄::Float64
    L₅::Float64
    L₆::Float64
    L₇::Float64
    L₈::Float64
    L₉::Float64
    L₁₀::Float64
    L₁₁::Float64
end
# 
# Outer constructors:
"""
Calculates the DLT coefficients from the camera calibration coefficients

Uses the equations in the appendix of Holland et al. 1997 to calculate the
Direct Linear Transformation coefficients directly from the physical parameters
of the camera's installation - location, orientation, magnification.
"""
function DLT(χ::ExtrinsicCalibrationParameters, ι::Union{CameraSpecs,IntrinsicCalibrationParameters}) 
    Rot = RotationMatrix(χ)
    LL = -( χ.x_c * Rot.m31 + χ.y_c * Rot.m32 + χ.z_c * Rot.m33 )
    abs(LL) < 1.0 ? @warn("LL has a small value! - results might be unstable.") : nothing
    # 
    L₁ = ( ι.u₀ * Rot.m31 -χ.f * Rot.m11 ) / (ι.λᵤ * LL)
    L₂ = ( ι.u₀ * Rot.m32 -χ.f * Rot.m12 ) / (ι.λᵤ * LL)
    L₃ = ( ι.u₀ * Rot.m33 -χ.f * Rot.m13 ) / (ι.λᵤ * LL)
    L₄ = -(L₁ * χ.x_c + L₂ * χ.y_c + L₃ * χ.z_c)
    L₅ = ( ι.v₀ * Rot.m31 -χ.f * Rot.m21 ) / (ι.λᵥ * LL)
    L₆ = ( ι.v₀ * Rot.m32 -χ.f * Rot.m22 ) / (ι.λᵥ * LL)
    L₇ = ( ι.v₀ * Rot.m33 -χ.f * Rot.m23 ) / (ι.λᵥ * LL)
    L₈ = -(L₅ * χ.x_c + L₆ * χ.y_c + L₇ * χ.z_c)
    L₉ = Rot.m31 / LL
    L₁₀ = Rot.m32 / LL
    L₁₁ = Rot.m33 / LL
    return DLT(L₁, L₂, L₃, L₄, L₅, L₆, L₇, L₈, L₉, L₁₀, L₁₁)
end
###############################################################################

###############################################################################
"""
    ExtrinsicCalibration(GCPuv, GCPxyz, χ, ι)

Returns the extrinsic camera calibration coefficients, calculated from a set of
GCPs in real-world and pixel-space coordinates, using the methodology of Holland
et al. 1997.
"""
function ExtrinsicCalibration(GCPuv::Array{Float64,2}, GCPxyz::Array{Float64,2},
                              χinitial::ExtrinsicCalibrationParameters,
                              ι::IntrinsicCalibrationParameters;
                              MaxIterations = 50,
                              PrintoutStats=true)
    # in practical experience if the method doesn't converge after about 30
    # iterations it probably will not converge within 101 either.
    χ = deepcopy(χinitial)      # this prevents modification of the original input
    nGCPs = minimum([size(GCPuv,1), size(GCPxyz,1)])
    A = fill(NaN, nGCPs*2,7)
    b = fill(NaN, nGCPs*2)
    CorrectionTerm = fill(Inf, 7)
    PrevCorrectionTerm = copy(CorrectionTerm)
    iteration,ContinueFlag = (1,true)
    while (ContinueFlag==true)
        for gcp in 1:nGCPs
            x,y,z = GCPxyz[gcp,:]
            u,v = GCPuv[gcp,:]
            F₀, G₀, Matrix_row1, Matrix_row2 = UpdateConstants(x, y, z, u, v, χ, ι)
            A[gcp,:]       = Matrix_row1
            A[gcp+nGCPs,:] = Matrix_row2
            b[gcp]         = -F₀
            b[gcp+nGCPs]   = -G₀
        end
        CorrectionTerm = A\b
        χ = ExtrinsicCameraCalibration.ExtrinsicCalibrationParameters(
            χ.φ + CorrectionTerm[1],
            χ.τ + CorrectionTerm[2],
            χ.σ + CorrectionTerm[3],
            χ.f + CorrectionTerm[4],
            χ.x_c + CorrectionTerm[5],
            χ.y_c + CorrectionTerm[6],
            χ.z_c + CorrectionTerm[7],
        )
        if (iteration > MaxIterations)
            @warn join(["exceeded maximum  iterations ($iteration)",
                        "max(|ε|)=$(map(y -> @sprintf("%.0e",y),maximum(abs.(CorrectionTerm))))",
                        "rms(ε₁-ε₀)=$(map(y -> @sprintf("%.0e",y),sqrt(mean(CorrectionTerm-PrevCorrectionTerm).^2)))"],
                       ", ")
            χ = ExtrinsicCameraCalibration.ExtrinsicCalibrationParameters(
                NaN, NaN, NaN, NaN, NaN, NaN, NaN)
            ContinueFlag = false
        end
        if ( (maximum(abs.(CorrectionTerm)) < 1e-10) || # convergenece conditions
             (sqrt(mean(CorrectionTerm-PrevCorrectionTerm).^2) <= 1e-10) )
            ContinueFlag = false
        end
        iteration <= 3 ? (ContinueFlag = true) : nothing # makes sure there have been at least 3 iterations
        ContinueFlag == true ? PrevCorrectionTerm = copy(CorrectionTerm) : nothing # store this iteration's results unless we're done - in that case keep the last iteration's results for reporting.
        iteration += 1
    end
    if PrintoutStats == true
        println(join(["Finished extrinsic calibration:",
                      "$iteration iterations",
                      "max(|ε|)=$(map(y -> @sprintf("%.0e",y),maximum(abs.(CorrectionTerm))))",
                      "rms(ε₁-ε₀)=$(map(y -> @sprintf("%.0e",y),sqrt(mean(CorrectionTerm-PrevCorrectionTerm).^2)))"],
                     ", "))
    end
    return χ
end
###############################################################################

##############################################################################
"""
Updates the system of equations using the constants calculated in the last iteration of ExtrinsicCalibration
"""
function UpdateConstants(x, y, z, u, v, χ, ι)
    # ---------- distances from the GCP to the camera:
    Δx  = f_Δx(x,χ)
    Δy  = f_Δy(y,χ)
    Δz  = f_Δz(z,χ)
    # ---------- undistorted, scale corrected image coordinates of the GCP:
    uˢ  = f_uˢ(u,ι)
    vˢ  = f_vˢ(v,ι)
    # ---------- elements of the 3x3 orthogonal rotation matrix (aka direction cosines):
    m11  = f_m11(χ)
    m12  = f_m12(χ)
    m13  = f_m13(χ)
    # --   
    m21  = f_m21(χ)
    m22  = f_m22(χ)
    m23  = f_m23(χ)
    # --   
    m31  = f_m31(χ)
    m32  = f_m32(χ)
    m33  = f_m33(χ)
    # ---------- results using rotation matrix:
    o = m11 * Δx + m12 * Δy + m13 * Δz
    p = m21 * Δx + m22 * Δy + m23 * Δz
    q = m31 * Δx + m32 * Δy + m33 * Δz 
    # ---------- bᵢⱼ are a set of constants used in LHS of the iterative calibration calculation:
    # Note that as calculated here they are multiplied by q.
    # b₁ⱼ are in the u coordinate direction
    qb11 = uˢ * (Δx*m32 -Δy*m31 ) + χ.f  * (Δx * m12 -Δy * m11 )
    qb12 = uˢ * (Δx * cos(χ.τ) * sin(χ.φ) + Δy * cos(χ.φ) * cos(χ.τ) + Δz * sin(χ.τ)) + χ.f  * (-Δx * sin(χ.φ) * sin(χ.τ) * sin(χ.σ) - Δy * cos(χ.φ) * sin(χ.τ) * sin(χ.σ) + Δz * cos(χ.τ) * sin(χ.σ))
    qb13 = χ.f * (Δx * m21 + Δy * m22 + Δz * m23 )
    qb14 = o                   
    qb15 = uˢ * m31 + χ.f * m11  
    qb16 = uˢ * m32 + χ.f * m12 
    qb17 = uˢ * m33 + χ.f * m13   
    # ----------
    # b₂ⱼ are in the v coordinate direction
    qb21 = vˢ * (Δx * m32 -Δy * m31 ) + χ.f  * (Δx * m22 -Δy * m21 ) 
    qb22 = vˢ * (Δx * cos(χ.τ) * sin(χ.φ) + Δy * cos(χ.φ) * cos(χ.τ) + Δz * sin(χ.τ)) + χ.f  * (-Δx * sin(χ.φ) * sin(χ.τ) * cos(χ.σ)-Δy * cos(χ.φ) * sin(χ.τ) * cos(χ.σ) + Δz * cos(χ.τ) * cos(χ.σ))
    qb23 = -χ.f * (Δx * m11 + Δy * m12 + Δz * m13 ) # 
    qb24 = p                   # 
    qb25 = vˢ * m31 + χ.f * m21
    qb26 = vˢ * m32 + χ.f * m22
    qb27 = vˢ * m33 + χ.f * m23
    ###############################################################################
    # F₀, G₀ form the RHS of the system of equations:
    F₀ = uˢ + o/q * χ.f 
    G₀ = vˢ + p/q * χ.f 
    ###############################################################################
    Matrix_row1 = [qb11, qb12, qb13, qb14, -qb15, -qb16, -qb17] ./ q
    Matrix_row2 = [qb21, qb22, qb23, qb24, -qb25, -qb26, -qb27] ./ q
    return F₀, G₀, Matrix_row1, Matrix_row2
end
###############################################################################



###############################################################################
# Below is a collection of helper functions used by ExtrinsicCalibration, and documented in Holland et al., 1997.
# ---------- distances from the GCP to the camera:
f_Δx(x,χ) = x - χ.x_c                  # [meters]
f_Δy(y,χ) = y - χ.y_c                  # [meters]
f_Δz(z,χ) = z - χ.z_c                  # [meters]
# ---------- undistorted, scale corrected image coordinates of the GCP:
f_uˢ(u,ι) = (u - ι.u₀)ι.λᵤ               # [pixels]
f_vˢ(v,ι) = (v - ι.v₀)ι.λᵥ               # [pixels]
# ---------- elements of the 3x3 orthogonal rotation matrix (aka direction cosines):
f_m11(χ) =  cos(χ.φ)cos(χ.σ) + sin(χ.φ)cos(χ.τ)sin(χ.σ)  # [dimensionless]
f_m12(χ) = -sin(χ.φ)cos(χ.σ) + cos(χ.φ)cos(χ.τ)sin(χ.σ) # [dimensionless]
f_m13(χ) =  sin(χ.τ)sin(χ.σ)                       # [dimensionless]
# --
f_m21(χ) = -cos(χ.φ)sin(χ.σ) + sin(χ.φ)cos(χ.τ)cos(χ.σ) # [dimensionless]
f_m22(χ) =  sin(χ.φ)sin(χ.σ) + cos(χ.φ)cos(χ.τ)cos(χ.σ)  # [dimensionless]
f_m23(χ) =  sin(χ.τ)cos(χ.σ)                       # [dimensionless]
# --
f_m31(χ) =  sin(χ.φ)sin(χ.τ)                       # [dimensionless]
f_m32(χ) =  cos(χ.φ)sin(χ.τ)                       # [dimensionless]
f_m33(χ) = -cos(χ.τ)                            # [dimensionless]
# ---------- results of rotation matrix(?):
f_o(x,y,z,χ) = f_m11(χ)*f_Δx(x,χ) + f_m12(χ)*f_Δy(y,χ) + f_m13(χ)*f_Δz(z,χ) # [meters]
f_p(x,y,z,χ) = f_m21(χ)*f_Δx(x,χ) + f_m22(χ)*f_Δy(y,χ) + f_m23(χ)*f_Δz(z,χ) # [meters]
f_q(x,y,z,χ) = f_m31(χ)*f_Δx(x,χ) + f_m32(χ)*f_Δy(y,χ) + f_m33(χ)*f_Δz(z,χ) # [meters]
# ---------- These functions are here for completeness, but aren't in use (due to UpdateConstants)
f_b11(x,y,z,u,v,χ,ι) = f_uˢ(u,ι)/f_q(x,y,z,χ) * ( f_Δx(x,χ)f_m32(χ) - f_Δy(y,χ)f_m31(χ) ) + χ.f/f_q(x,y,z,χ) * ( f_Δx(x,χ)f_m12(χ) - f_Δy(y,χ)f_m11(χ) )     # ∂F/∂χ.φ ; (pixels/meters)*(meters) + => [pixels]
f_b12(x,y,z,u,v,χ,ι) = f_uˢ(u,ι)/f_q(x,y,z,χ) * ( f_Δx(x,χ)cos(χ.τ)sin(χ.φ) + f_Δy(y,χ)cos(χ.φ)cos(χ.τ) + f_Δz(z,χ)sin(χ.τ) ) + χ.f/f_q(x,y,z,χ) * ( -f_Δx(x,χ)sin(χ.φ)sin(χ.τ)sin(χ.σ) - f_Δy(y,χ)cos(χ.φ)sin(χ.τ)sin(χ.σ) + f_Δz(z,χ)cos(χ.τ)sin(χ.σ) )  # ∂F/∂χ.τ ; [pixels/meters]*[meters] => [pixels]
f_b13(x,y,z,u,v,χ,ι) = χ.f/f_q(x,y,z,χ) * ( f_Δx(x,χ)f_m21(χ) + f_Δy(y,χ)f_m22(χ) + f_Δz(z,χ)f_m23(χ) )                                # ∂F/∂χ.σ ; [pixels/meters]*[meters] => [pixels]
f_b14(x,y,z,u,v,χ,ι) = f_o(x,y,z,χ)/f_q(x,y,z,χ)                                                                      # ∂F/∂f ; [meters/meters] => [dimensionless]
f_b15(x,y,z,u,v,χ,ι) = f_uˢ(u,ι)/f_q(x,y,z,χ) * f_m31(χ) + χ.f/f_q(x,y,z,χ) * f_m11(χ)                                             # ∂F/∂χ.x_c ; [pixels/meter]
f_b16(x,y,z,u,v,χ,ι) = f_uˢ(u,ι)/f_q(x,y,z,χ) * f_m32(χ) + χ.f/f_q(x,y,z,χ) * f_m12(χ)                                             # ∂F/∂χ.y_c ; [pixels/meter]
f_b17(x,y,z,u,v,χ,ι) = f_uˢ(u,ι)/f_q(x,y,z,χ) * f_m33(χ) + χ.f/f_q(x,y,z,χ) * f_m13(χ)                                             # ∂F/∂χ.z_c ; [pixels/meter]
# ----------
f_b21(x,y,z,u,v,χ,ι) = f_vˢ(v,ι)/f_q(x,y,z,χ) * ( f_Δx(x,χ)f_m32(χ) - f_Δy(y,χ)f_m31(χ) ) + χ.f/f_q(x,y,z,χ) * ( f_Δx(x,χ)f_m22(χ) - f_Δy(y,χ)f_m21(χ) )     # ∂G/∂χ.φ
f_b22(x,y,z,u,v,χ,ι) = f_vˢ(v,ι)/f_q(x,y,z,χ) * ( f_Δx(x,χ)cos(χ.τ)sin(χ.φ) + f_Δy(y,χ)cos(χ.φ)cos(χ.τ) + f_Δz(z,χ)sin(χ.τ) ) + χ.f/f_q(x,y,z,χ) * ( -f_Δx(x,χ)sin(χ.φ)sin(χ.τ)cos(χ.σ) - f_Δy(y,χ)cos(χ.φ)sin(χ.τ)cos(χ.σ) + f_Δz(z,χ)cos(χ.τ)cos(χ.σ) )  # ∂G/∂χ.τ ; [pixels/meters]*[meters] => [pixels]
f_b23(x,y,z,u,v,χ,ι) = -χ.f/f_q(x,y,z,χ) * ( f_Δx(x,χ)f_m11(χ) + f_Δy(y,χ)f_m12(χ) + f_Δz(z,χ)f_m13(χ) )                               # ∂G/∂χ.σ ; [pixels/meters]*[meters] => [pixels]
f_b24(x,y,z,u,v,χ,ι) = f_p(x,y,z,χ)/f_q(x,y,z,χ)                                                                      # ∂G/∂f ; [meters/meters] => [dimensionless]
f_b25(x,y,z,u,v,χ,ι) = f_vˢ(v,ι)/f_q(x,y,z,χ) * f_m31(χ) + χ.f/f_q(x,y,z,χ) * f_m21(χ)                                             # ∂G/∂χ.x_c ; [pixels/meter]
f_b26(x,y,z,u,v,χ,ι) = f_vˢ(v,ι)/f_q(x,y,z,χ) * f_m32(χ) + χ.f/f_q(x,y,z,χ) * f_m22(χ)                                             # ∂G/∂χ.y_c ; [pixels/meter]
f_b27(x,y,z,u,v,χ,ι) = f_vˢ(v,ι)/f_q(x,y,z,χ) * f_m33(χ) + χ.f/f_q(x,y,z,χ) * f_m23(χ)                                             # ∂G/∂χ.z_c ; [pixels/meter]
###############################################################################
# Functions for the RHS of the system of ef_quations:
f_F₀(x,y,z,u,v,χ,ι) = f_uˢ(u,ι)+f_o(x,y,z,χ)*χ.f/f_q(x,y,z,χ)
f_G₀(x,y,z,u,v,χ,ι) = f_vˢ(v,ι)+f_p(x,y,z,χ)*χ.f/f_q(x,y,z,χ)
###############################################################################



###############################################################################
###############################################################################
###############################################################################
end                             # end of module
###############################################################################
###############################################################################
###############################################################################



Main.ExtrinsicCameraCalibration

In [28]:
# IRQIV submodule 4
###############################################################################
# 
# ImageRectification.jl in /home/seth/.julia/dev/IRQIV/src/
# 
# Created: 2023-04-27 10:27:30 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-27 12:51:58 seth>
# 
###############################################################################

###############################################################################
module ImageRectification
###############################################################################

###############################################################################
using StaticArrays  # StaticArrays accelerate uvz2xy by two orders of magnitude!
import IRQIV: DLT, ExtrinsicCalibrationParameters
###############################################################################

###############################################################################
export
    uvz2xy,
    xyz2uv
###############################################################################

###############################################################################
# Pixel to physical coordinate transformation:
uvz2xy(u::Number, v::Number, z::Number, dlt::DLT) = Amatrix(u, v, z, dlt) \ bvector(u, v, z, dlt)
Amatrix(u, v, z, dlt::DLT) = SMatrix{2,2}( u*dlt.L₉-dlt.L₁,  v*dlt.L₉-dlt.L₅, u*dlt.L₁₀-dlt.L₂, v*dlt.L₁₀-dlt.L₆ )
bvector(u, v, z, dlt::DLT) = SVector( dlt.L₄-u + (dlt.L₃-u*dlt.L₁₁)*z, dlt.L₈-v + (dlt.L₇-v*dlt.L₁₁)*z )
#
function uvz2xy(u::AbstractArray, v::AbstractArray, z::AbstractArray, dlt::DLT)
    x = similar(u, Float64)
    y = similar(x)
    for i in eachindex(u)
        x[i],y[i] = uvz2xy(u[i], v[i], z[i], dlt)
    end
    return x, y
end
uvz2xy(u::AbstractArray, v::AbstractArray, z::Number, dlt::DLT) = uvz2xy(u, v, fill!(similar(u, eltype(z)), z), dlt)
uvz2xy(uv::AbstractArray, z, dlt::DLT) = uvz2xy(uv[:,1], uv[:,2], z, dlt)
uvz2xy(uvz::AbstractArray, dlt::DLT) = uvz2xy(uvz[:,1], uvz[:,2], uvz[:,3], dlt)
###############################################################################


###############################################################################
# Physical to pixel coordinate transformation:
xyz2uv(x, y, z, dlt::DLT) = (u_numerator(x, y, z, dlt), v_numerator(x, y, z, dlt)) ./ uv_denominator(x, y, z, dlt)
u_numerator(x, y, z, dlt::DLT) = (dlt.L₁*x + dlt.L₂*y  + dlt.L₃*z  + dlt.L₄)
v_numerator(x, y, z, dlt::DLT) = (dlt.L₅*x + dlt.L₆*y  + dlt.L₇*z  + dlt.L₈)
uv_denominator(x, y, z, dlt::DLT) = (dlt.L₉*x + dlt.L₁₀*y + dlt.L₁₁*z + 1 )
#
function xyz2uv(x::AbstractArray, y::AbstractArray, z::AbstractArray, dlt::DLT)
    u = similar(x, Float64)
    v = similar(u)
    for i in eachindex(u)
        u[i],v[i] = xyz2uv(x[i], y[i], z[i], dlt)
    end
    return u, v
end
xyz2uv(x::AbstractArray, y::AbstractArray, z::Number, dlt::DLT) = xyz2uv(x, y, fill!(similar(x, eltype(z)), z), dlt)
xyz2uv(xy::AbstractArray, z, dlt::DLT) = xyz2uv(xy[:,1], xy[:,2], z, dlt)
xyz2uv(xyz::AbstractArray, dlt::DLT) = xyz2uv(xyz[:,1], xyz[:,2], xyz[:,3], dlt)
###############################################################################


###############################################################################
end                             # end of module
###############################################################################



Main.ImageRectification

In [29]:
# IRQIV submodule 5
###############################################################################
# 
# ThreadedMQD.jl in /home/seth/Dropbox-sas262/Dropbox/Julia/QIV-working/pattern-tracking/
# 
# Created: 2023-04-18 17:00:31 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-27 08:54:43 seth>
# 
###############################################################################


###############################################################################
"""
TODO: compare performance using MKL: FFTW.set_provider!("mkl")
TODO: wrap the subwindow iteration in a function
TODO: wrap the image (I1,I2) iteration in a function.
TODO: consider additional padding of g1star, g2star, to a 2^n size, for faster fft
TODO: investigate performance with different data types: views, PaddedArrays,
      simple arrays instead of structs, etc.
TODO: look into @fastmath, @simd, in addition to @turbo
TODO: implement "ManualMQD" - do actual abs2(g1-g2) in a loop, to compare
      results and performance.
TODO: Add offset array like syntax for subwindows.
TODO: Q isn't really necessary, it is just a view into I2.QfullROI!!
TODO: similarly, g2star isn't really necessary, it is just a view into I2.data
TODO: and, g1star is really just a zero-padded view into I1
TODO: change g1star definition, such that update_g1star! only needs the
      subwindow corner as input, and not the subwindow offsets as well
DONE:
DONE: Add plan_fft, plan_ifft!
DONE: reduce temporary arrays by breaking fft/ifft cycle into steps, and
      reusing the same variable.
DONE: look into r2r (real-only fft) -> now implemented with rfft
TODO: wrap the initialization in a function (g1star, g2star, R, etc.)
"""
###############################################################################




###############################################################################
using LoopVectorization
using FFTW
using Statistics
#
import Base: sum
###############################################################################



###############################################################################
# I1: (converts pixels to Matrix{Float64})
initialize_I1(::T, pixels, mean_pixel_vals=zeros(eltype(pixels), size(pixels))) where T<:Number =
    DataStruct{Matrix{Float64}}(pixels .- mean_pixel_vals) # converts pixel data to Float64s if necessary
initialize_I1(pixels, mean_pixel_vals=zeros(eltype(pixels), size(pixels))) = initialize_I1(Float64, pixels, mean_pixel_vals)
# I2:
# ideal:
function initialize_I2(pixels, M, N, rhoI, rhoJ, subwindow_topleft_corners::CartesianIndices)
    Qmini = first(subwindow_topleft_corners)[1] - rhoI
    Qminj = first(subwindow_topleft_corners)[2] - rhoJ
    Qmaxi = last(subwindow_topleft_corners)[1] + (rhoI-1)
    Qmaxj = last(subwindow_topleft_corners)[2] + (rhoJ-1)
    # send zeros as initial QfullROI:
    @assert Qmini > 0
    @assert Qminj > 0
    @assert Qmaxi + (N-1) <= size(pixels,1)
    @assert Qmaxj + (M-1) <= size(pixels,2)
    I2(pixels, zeros(eltype(pixels), size(pixels)), Qmini, Qmaxi, Qminj, Qmaxj, M, N)
end
# but allow Q limits to be defined manually. In this case we'll require the
# mean - up to the user to create zeros if necessary.
function initialize_I2(pixels,
                 Qmini, Qmaxi,
                 Qminj, Qmaxj,
                 M, N)
    I2(pixels, Qmini, Qmaxi, Qminj, Qmaxj, M, N)
end
initialize_g1star(::Type{T}, M, N, rhoI, rhoJ) where {T<:Number} =
    IndexedDataStruct(zeros(T, N+2rhoI, M+2rhoJ),
                            CartesianIndices((rhoI .+ (1:N), rhoJ .+ (1:M))))
initialize_g1star(M, N, rhoI, rhoJ) = initialize_g1star(Float64, M, N, rhoI, rhoJ) # default to Float64
# 
initialize_g2star(::Type{T}, M, N, rhoI, rhoJ) where {T<:Number} =
    IndexedDataStruct(zeros(T, N+2rhoI, M+2rhoJ),
                            CartesianIndices( ( (-rhoI):(-1+N+rhoI), (-rhoJ):(-1+M+rhoJ) ) ))
initialize_g2star(M, N, rhoI, rhoJ) = initialize_g2star(Float64, M, N, rhoI, rhoJ) # default to Float64
# 
initialize_Dstar(rhoI,rhoJ) = Dstar(zeros(Float64, 2rhoI,2rhoJ),
                              rhoI,rhoJ,
                              rhoI |> ρ -> -ρ:(ρ-1), # div is integer division
                              rhoJ |> ρ -> -ρ:(ρ-1),
                              )
#
initialize_Q(::Type{T}, rhoI, rhoJ) where {T<:Number} = IndexedDataStruct(zeros(T, 2rhoI, 2rhoJ),
                                                            CartesianIndices(( (-rhoI:(rhoI-1)), (-rhoJ:(rhoJ-1)) )))
initialize_Q(rhoI, rhoJ) = initialize_Q(Float64, rhoI, rhoJ)
###############################################################################

###############################################################################
""" Simple container for an array of data, like I1 or mean_image. """
struct DataStruct{T}
    data::T
end
###############################################################################

###############################################################################
""" Container for data (e.g., g1star, g2star), and indices that are often used
with these data """
struct IndexedDataStruct{T}
    data::T
    inds::CartesianIndices
end
###############################################################################

###############################################################################

# R is called Φ in Gui & Merzkirch (2000), and other PIV literature, and should
# probably be renamed to that.
""" R contains the FFT'd portion of the MQD calculation, calculated from g1star and g2star. """
struct R
    data::Matrix{Float64}
    inds::Matrix{CartesianIndex{2}}
    p_rfft::FFTW.rFFTWPlan      # expects real input
    p_irfft::AbstractFFTs.ScaledPlan
    temp1::Matrix{ComplexF64}
    temp2::Matrix{ComplexF64}
    temp3::Matrix{Float64}
end
#
function initialize_R(M, N, rhoI, rhoJ)
    data = zeros(Float64, 2rhoI, 2rhoJ)
    temp1 = zeros(ComplexF64, div(N+2rhoI, 2)+1, M+2rhoJ)
    temp2 = similar(temp1)
    temp3 = zeros(Float64, N+2rhoI, M+2rhoJ)       # temp3 can be repalced with g2star.data
    # these `inds` have fftshift baked in:
    inds = FFTW.fftshift(CartesianIndices(temp3))[CartesianIndices((Int(N/2) .+ (1:2rhoI), Int(M/2) .+ (1:2rhoJ)))]
    p_rfft = FFTW.plan_rfft(zeros(N+2rhoI, M+2rhoJ))
    p_irfft = FFTW.plan_irfft(temp1, N+2rhoI)
    # 
    R(data, inds, p_rfft, p_irfft, temp1, temp2, temp3)
end
# 
function update_R!(R::R, g1star::IndexedDataStruct, g2star::IndexedDataStruct)
    # @turbo doesn't (currently) work with complex number opertations, or with
    # Matrix{CartesianIndex{2}} indices
    @inbounds @fastmath FFTW.mul!(R.temp2, R.p_rfft, g2star.data)
    @inbounds @fastmath FFTW.mul!(R.temp1, R.p_rfft, g1star.data)
    @inbounds @fastmath R.temp2 .*= conj( R.temp1 )
    @inbounds @fastmath FFTW.mul!( R.temp3, R.p_irfft, R.temp2 )
    @inbounds @fastmath R.data .= R.temp3[R.inds]
end
# 
###############################################################################

###############################################################################
""" struct for I2 image, including some parameters used by MQD: QfullROI
contains the square of pixel intensity values, only within the region defined by
Qmin:Qmax, N,M define the subwindow dimensions """
struct I2{T}
    data::T
    QfullROI::T
    Qmini::Int
    Qmaxi::Int
    Qminj::Int
    Qmaxj::Int
    M::Int
    N::Int
    I2(data, QfullROI, Qmini, Qmaxi, Qminj, Qmaxj, M, N) =
        if all([ Qmini > 0, Qminj > 0, Qmaxi+N <= size(data,1), Qmaxj+M <= size(data,2)])
            new{typeof(data)}(data, QfullROI, Qmini, Qmaxi, Qminj, Qmaxj, M, N)
        else
            error("bad input! Q limits out of bounds!")
        end
end
###############################################################################

###############################################################################
""" Calculates Q over the ROI """
function update_QfullROI!(I2::I2{Matrix{Float64}})
    @turbo for J in CartesianIndices((I2.Qmini:I2.Qmaxi, I2.Qminj:I2.Qmaxj))
        tmp = zero(eltype(I2.QfullROI))
        for I ∈ CartesianIndices((I2.N,I2.M))
            tmp += abs2(I2.data[I + J])
        end
        I2.QfullROI[J] = tmp
    end
end
###############################################################################


###############################################################################
function update_data!(oldstruct::Union{DataStruct, IndexedDataStruct}, newdata)
    # @turbo for i in eachindex(newdata)
    @inbounds for i in eachindex(newdata)
        oldstruct.data[i] = newdata[i]
    end
end
###############################################################################

###############################################################################
update_I1!(I1::DataStruct, new_image) = update_data!(I1, new_image)
function update_I2!(I2::I2, newdata)
    # @turbo for i in eachindex(newdata)
    @inbounds for i in eachindex(newdata)
        I2.data[i] = newdata[i]
    end
    update_QfullROI!(I2)
end
###############################################################################

###############################################################################
""" g1star contains g1---the subwindow pattern from I1---zero padded by rhoI or
rhoJ zeors on each side, in order to match the size of g2star and have the
correct dimensions for the fft calculation. g1star.data's dimensions are
therefore Nstar×Mstar, or (rhoI+N+rhoI)×(rhoJ+M+rhoJ), of which only the central
N×M pixels contain data, and the rest are zeros. """
function update_g1star!(g1star::IndexedDataStruct, I1::DataStruct, subwininds)
    g1star.data[g1star.inds] .= I1.data[subwininds] 
end
###############################################################################

###############################################################################
""" g2star contains (g2), the (N×M) subwindow pattern from I2, but extended by
rhoI pixels on the top and bottom, and rhoJ pixels on the right and left, to
accomodate the search region. """
function update_g2star!(g2star::IndexedDataStruct, I2::I2, subwin_corner)
    g2star.data .= I2.data[ subwin_corner .+ g2star.inds ]
end
###############################################################################

###############################################################################
""" Q contains the sum of squared pixel intensity values in I2, over an area the
size of a N×M subwindow extending from the coordinate in Q. That is, Q[i,j]
contains sum(I2[i.+(0:N-1), j.+(0:M-1)].^2) """
function update_Q!(Q::IndexedDataStruct, I2::I2, subwin_corner)
    Q.data .= I2.QfullROI[ subwin_corner .+ Q.inds ]
end
###############################################################################

###############################################################################
""" structure containing Q - 2R, which is the "mqd" surface, the minima of which
is at the location that represents the pixel-shift corresponding to the
displacment of the tracked pattern in the subwindow.

ishifts and jshifts are currently defined as StepRangeLen, but perhaps this can
be simplified.
"""
struct Dstar
    data::Matrix{Float64}
    rhoI::Int
    rhoJ::Int
    ishifts::StepRangeLen
    jshifts::StepRangeLen
end
###############################################################################

###############################################################################
""" updates Dstar, using the current values of Q and R. Note that Q is really
just a view into I2.QfullROI, so perhaps it should be eliminated as a variable,
and this function changed to access I2 instead. """
function update_Dstar!(dstar::Dstar, Q::IndexedDataStruct, R)
    @turbo for i in eachindex(dstar.data)
        dstar.data[i] = Q.data[i] - 2R.data[i]
    end
end
function update_Dstar!(dstar::Dstar, Q, R)
    @turbo for i in eachindex(dstar.data)
        dstar.data[i] = Q[i] - 2R.data[i]
    end
end
###############################################################################

###############################################################################
""" faster implementation of Base.argmin() """
function argmin_turbo(x)
    indmin = 0
    minval = typemax(eltype(x))
    @turbo for i ∈ eachindex(x)
        newmin = x[i] < minval
        minval = newmin ? x[i] : minval
        indmin = newmin ?   i  : indmin
    end
    # return indmin
    return CartesianIndices(x)[indmin]
end
###############################################################################

###############################################################################
""" structure containing the integer and subpixel parts of a pixel displecement"""
struct MQDminima
    intshift::Int
    subpixshift::Float64
end
sum(x::MQDminima) = sum([x.intshift, x.subpixshift])
###############################################################################

###############################################################################
""" finds the sub-pixel fit of the minima of the surface dstar, using a 3-point
Gaussian estimator. (Based on Willert & Gharib's method?) """
function subpixel_fit_minima(dstar::Dstar)
    i,j = Tuple(argmin_turbo(dstar.data))
    if  1 < i  && i < 2dstar.rhoI
        @inbounds γ₁,γ₂,γ₃ = dstar.data[[i-1,i,i+1], j] .|> Complex
        iₛ = ( log(γ₁)-log(γ₃) ) /
            2( log(γ₃) -2log(γ₂) + log(γ₁) ) |> real
    else            
        iₛ = 0.0
    end
    if 1 < j && j < 2dstar.rhoJ
        @inbounds ξ₁,ξ₂,ξ₃ = dstar.data[i, [j-1,j,j+1]] .|> Complex
        jₛ = ( log(ξ₁)-log(ξ₃) ) /
            2( log(ξ₃) -2log(ξ₂) + log(ξ₁) ) |> real
    else            
        jₛ = 0.0
    end
    return @inbounds MQDminima(dstar.ishifts[i], iₛ), MQDminima(dstar.jshifts[j], jₛ)
end
###############################################################################

###############################################################################
""" impelement an MQD function without FFT acceleration, for comparison and
testing purposes.  """
function ManualMQD(i,j, N, M, rhoI, rhoJ) end
###############################################################################



###############################################################################
"""
Initializes variables needed for MQD analysis.

If Julia is running in a multithreaded environment, initializes in parallel for
all available threads.

Inputs are the images, subwindow locations in pixel coordinates, and basic
MQD_parameters of subwindow size (N,M) and search radii (rhoI, rhoJ).

Output is a Tuple containing all the initialized components required by
perform_MQD().
"""
function initialize_MQD(I1images, I2images, subwincorners, N, M, rhoI, rhoJ)
    mean_image = DataStruct(mean(I1images.Frames, dims=3) |> x->dropdims(x,dims=3))
    (mean_image = mean_image,
     subwinoffsets = CartesianIndices(( (1:N).-1, (1:M).-1 )),
     Multithread_I1 = [DataStruct(Float64.(I1images.Frames[:,:,i]) .- mean_image.data) for i in 1:Threads.nthreads() ],
     Multithread_I2 = [initialize_I2(I2images.Frames[:,:,i] .- mean_image.data, M, N, rhoI, rhoJ, subwincorners) for i in 1:Threads.nthreads() ],
     Multithread_g1star = [ initialize_g1star(Float64, M, N, rhoI, rhoJ) for i in 1:Threads.nthreads() ],
     Multithread_g2star = [ initialize_g2star(Float64, M, N, rhoI, rhoJ) for i in 1:Threads.nthreads() ],
     Multithread_R = [ initialize_R(M, N, rhoI, rhoJ) for i in 1:Threads.nthreads() ],
     Multithread_Q = [ initialize_Q(Float64, rhoI, rhoJ) for i in 1:Threads.nthreads() ],
     Multithread_Dstar = [ initialize_Dstar(rhoI, rhoJ) for i in 1:Threads.nthreads() ],
     )
end
###############################################################################


###############################################################################
"""
Performs MQD pattern tracking analysis over all image pairs contained in
I1images and I2images, at the subwindow locations defined by subwincorners. Uses
the variables contained in MQD_variables for this (these were likely initialized
by `initialize_MQD()`). MQD_parameters are only required explicitely here to
format the output performance message, since these parameters are already
implicitly contained in the MQD_variables.

Returns two arrays with the pixel-shifts in the i and j pixel directions,
(dimensions 1 and two of the arrays), with the third dimension of the array
corresponding to image pairs. Also returns a formatted string describing the
computational performance.
"""
function perform_MQD(I1images, I2images, subwincorners, MQD_variables, MQD_parameters)
    vars = MQD_variables
    # 
    subpix_shifts_i = fill(NaN, size(subwincorners)..., length(I1images));
    subpix_shifts_j = fill(NaN, size(subwincorners)..., length(I1images));
    # 
    shifts_ci = CartesianIndices(subwincorners)
    # performance_stats = @timed Threads.@threads for imgpair_ind in 1
    # @warn("only analyzing one image pair!")
    # performance_stats = @timed Threads.@threads for imgpair_ind in 1:size(I1images,3)
    performance_stats = @timed Threads.@threads for imgpair_ind in ProgressBar(1:size(I1images,3))
        _id = Threads.threadid()
        ## Subtract mean image:
        update_I1!(vars.Multithread_I1[_id], I1images.Frames[:,:,imgpair_ind] .- vars.mean_image.data)
        update_I2!(vars.Multithread_I2[_id], I2images.Frames[:,:,imgpair_ind] .- vars.mean_image.data)
        for subwin_ind in 1:length(subwincorners)
            # @show _id, imgpair_ind, subwin_ind
            ci = subwincorners[subwin_ind]
            update_g1star!(vars.Multithread_g1star[_id], vars.Multithread_I1[_id], ci .+ vars.subwinoffsets)
            update_g2star!(vars.Multithread_g2star[_id], vars.Multithread_I2[_id], ci) 
            update_R!(vars.Multithread_R[_id], vars.Multithread_g1star[_id], vars.Multithread_g2star[_id])
            update_Q!(vars.Multithread_Q[_id], vars.Multithread_I2[_id], ci)
            update_Dstar!(vars.Multithread_Dstar[_id], vars.Multithread_Q[_id], vars.Multithread_R[_id])
            # 
            _shifti, _shiftj = subpixel_fit_minima(vars.Multithread_Dstar[_id])
            # 
            subpix_shifts_i[shifts_ci[subwin_ind],imgpair_ind]  = sum(_shifti)
            subpix_shifts_j[shifts_ci[subwin_ind],imgpair_ind]  = sum(_shiftj)
        end
    end
    performance_message = format_outputstring(performance_stats,
                                              subpix_shifts_i, MQD_parameters,
                                              "Performance stats (MQD loop only)")
    @info performance_message
    return subpix_shifts_i, subpix_shifts_j, performance_message
end
###############################################################################



###############################################################################
format_line(label, value) = string(rpad(label, 20),lpad(value, 20),"\n")
function format_outputstring(performance_stats,
                             subpix_shifts_i, MQD_parameters,
                             headline="Performance stats (MQD loop only)")
    buf = IOBuffer()
    Sys.cpu_summary(buf)
    cpu_info = String(take!(buf))
    outputstring = join([headline, "\n",
                         format_line("time [s]", performance_stats.time),
                         format_line("subwins/s", length(subpix_shifts_i[:,:,1])/performance_stats.time),
                         format_line("allocations [GB]", performance_stats.bytes * 1e-9|>x->round(x,digits=2)),
                         format_line("allocations [MB/subwin]", (performance_stats.bytes*1e-6/length(subpix_shifts_i[:,:,1]))|>x->round(x,digits=2)),
                         format_line("time [m]", performance_stats.time / 60|>x->round(x,digits=2)),
                         format_line("total n subwindows", length(subpix_shifts_i)),
                         format_line("n image pairs", size(subpix_shifts_i,3)),
                         format_line("n subwindows in FOV", length(subpix_shifts_i[:,:,1])),
                         format_line("size(subwindows)", size(subpix_shifts_i[:,:,1])),
                         format_line("M, N", (MQD_parameters.M, MQD_parameters.N)),
                         format_line("rhoI, rhoJ", (MQD_parameters.rhoI,MQD_parameters.rhoJ)),
                         format_line("allocations [B]", performance_stats.bytes),
                         format_line("allocations [MB]", performance_stats.bytes * 1e-6|>x->round(x,digits=2)),
                         format_line("gc time [s]", performance_stats.gctime),
                         format_line("gc stats  ", performance_stats.gcstats),
                         format_line("number of threads  ", Threads.nthreads()),
                         "Running on $(Sys.CPU_NAME) CPU specs:\n",
                         cpu_info,
                         ], "")
    return outputstring
end
###############################################################################


LoadError: error in method definition: function IRQIV.initialize_MQD must be explicitly imported to be extended

In [30]:
# IRQIV submodule 6
###############################################################################
# 
# RemoveImageDistortion.jl in ~/.julia/dev/IRQIV/src/
# 
# Created: 2023-03-01 15:19:12 by seth@sas262-Arch
# last changed Time-stamp: <2023-04-26 13:11:21 seth>
# 
# used to be RemoveRadialDistortionLib.jl
# 
###############################################################################

###############################################################################
# import IRQIV: CameraSpecs
###############################################################################

###############################################################################
# RemoveRadialDistortion to move pixels to where they should be,
# SimulateRadialDistortion to map undirstorted coordinates to where they will
# appear on a distorted image.
###############################################################################

###############################################################################
# -----------------------------------------------------------------------------
function RemoveRadialDistortion(u_d::Number, v_d::Number, Degrees, Coeffs)
    up,vp = RemoveRadialDistortion([u_d], [v_d], Degrees, Coeffs)
    return up[1],vp[1]
end
# -----------------------------------------------------------------------------
function RemoveRadialDistortion(u_d::Array, v_d::Array, Degrees, Coeffs, u0, v0)
    ρ_d = Array{Float64}(undef, size(u_d))
    Θ_d = Array{Float64}(undef, size(u_d))
    for i in eachindex(u_d)
        ρ_d[i] = abs((u_d[i]-u0) + (v_d[i]-v0)im)
        Θ_d[i] = angle((u_d[i]-u0) + (v_d[i]-v0)im)
    end
    # Calculate the Δρ correction needed at each point:
    Δρ = Polyvalc(ρ_d, Coeffs, Degrees)
    Δu = cos.(Θ_d) .* Δρ
    Δv = sin.(Θ_d) .* Δρ
    # Apply the correction to ρ and convert back to pixel coordinates:
    u_p = u_d - Δu
    v_p = v_d - Δv
    return u_p,v_p 
end
# -----------------------------------------------------------------------------
function RemoveRadialDistortion(u_d::Array, v_d::Array, Camera::CameraSpecs)
    Coeffs = Camera.radial_distortion_poly_coeffs
    Degrees = Camera.radial_distortion_poly_degrees
    # Convert the *observed* pixel to polar coordinates:
    ρ_d = abs.((u_d .- Camera.u₀) .+ (v_d .- Camera.v₀)im)
    Θ_d = angle.((u_d .- Camera.u₀) .+ (v_d .- Camera.v₀)im)
    # Calculate the Δρ correction needed at each point:
    Δρ = Polyvalc(ρ_d, Coeffs, Degrees)
    Δu = cos.(Θ_d) .* Δρ
    Δv = sin.(Θ_d) .* Δρ
    # Apply the correction to ρ and convert back to pixel coordinates:
    u_p = u_d - Δu
    v_p = v_d - Δv
    return u_p,v_p
end
###############################################################################


###############################################################################
function SimulateRadialDistortion(uv_p::Array, Camera::CameraSpecs)
    Coeffs = Camera.radial_distortion_poly_coeffs
    Degrees = Camera.radial_distortion_poly_degrees
    return SimulateRadialDistortion(uv_p[:,1], uv_p[:,2], Degrees, Coeffs)
end
function SimulateRadialDistortion(u_p::Number, v_p::Number, Camera::CameraSpecs)
    return SimulateRadialDistortion([u_p], [v_p], Camera)
end
function SimulateRadialDistortion(u_p::Array, v_p::Array, Camera::CameraSpecs)
    Coeffs = Camera.radial_distortion_poly_coeffs
    Degrees = Camera.radial_distortion_poly_degrees
    return SimulateRadialDistortion(u_p::Array, v_p::Array, Degrees, Coeffs)
end
function SimulateRadialDistortion(u_p::Array, v_p::Array, Degrees, Coeffs, u0, v0)
    # Convert the *observed* pixel to polar coordinates:
    ρ_p = Array{Float64}(undef, size(u_p))
    Θ_p = Array{Float64}(undef, size(u_p))
    for i in eachindex(u_p)
        ρ_p[i] = abs((u_p[i]-u0) + (v_p[i]-v0)im)
        Θ_p[i] = angle((u_p[i]-u0) + (v_p[i]-v0)im)
    end
    # Calculate the Δρ correction needed at each point:
    Δρ = Polyvalc(ρ_p, Coeffs, Degrees)
    Δu = cos.(Θ_p) .* Δρ
    Δv = sin.(Θ_p) .* Δρ
    # Apply the correction to ρ and convert back to pixel coordinates:
    u_d = u_p + Δu
    v_d = v_p + Δv
    return u_d,v_d
end
###############################################################################


###############################################################################
# Polyfitc is similar to polyfit, but allows choosing powers (5,3,1, etc.)
# (based on Matlab file exchange Polyfitc calculations):
function Polyfitc(X,Y,Degrees)
    lsqM = ones(length(X), length(Degrees))
    for n = 1:length(Degrees)
        lsqM[:, n] = X.^Degrees[n]
    end
    Coeffs = (lsqM \ Y)'
    return Coeffs[:]
end
# ----------
function Polyvalc(X, Coeffs, Degrees)
    val(x,Coeffs,Degrees) = sum( [ Coeffs[i]*x^(Degrees[i]) for i in 1:length(Coeffs) ] )
    Y = [ val(x,Coeffs,Degrees) for x in X ]
end
###############################################################################



Polyvalc (generic function with 1 method)

In [31]:
###############################################################################
# 
# CameraCalibrationHelpers.jl in /home/seth/.julia/dev/IRQIV/src/
# 
# Created: 2023-04-27 10:24:33 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-27 13:33:01 seth>
# 
###############################################################################

###############################################################################
import PyPlot as plt
using Statistics
###############################################################################

###############################################################################
# Pkg.activate(expanduser("~/.julia/dev/IRQIV/")) # path to IRQIV directory
# Pkg.activate(raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\IRQIV.jl\src\IRQIV")
# using IRQIV
###############################################################################


###############################################################################
function GraphicallyExploreInitialCalibrationGuess(GCPuv, GCPxyz, χ, ι,
                                                   frame=fill(NaN, ι.v₀*2 - 1 |> Int, ι.u₀*2 - 1 |> Int))
    fig,ax = plt.subplots(1,2, squeeze=false)
    [ axh.set_aspect("equal") for axh in ax ]
    # 
    PlotGCPxyFromExtrinsicParameters(GCPuv, GCPxyz, χ, ι; ax=ax[1,1])
    ax[1,1].set_title("xy calculated from uvz")
    ax[1,2].imshow(frame; extent=[1,size(frame,2), size(frame,1), 1], alpha=0.3)
    PlotGCPuvFromExtrinsicParameters(GCPuv, GCPxyz, χ, ι; ax=ax[1,2])
    ax[1,2].invert_yaxis()
    ax[1,2].set_title("uv calculated from xyz")
    # 
    # ax[1,1].set_ylabel("using χ")
    #
    fig.suptitle(join(["red is calculated, black is provided\n\n",
                       sprint(dump, χ; context=:compact=>true)]))
end
###############################################################################

###############################################################################
"""
Calculates GCP x,y (physical) coordinates from provided u,v (pixel coordinates),
and provided intrinsic and extrinsic calibration parameters.

Plots calculated and provided GCP xy coordinates, and calculates the mean
distance between each GCP in the provided, and calculated, coordinate sets
"""
function PlotGCPxyFromExtrinsicParameters(GCPuv, GCPxyz,
                                          χ::ExtrinsicCalibrationParameters,
                                          ι::IntrinsicCalibrationParameters;
                                          ax=plt.gca())
    calcuatedGCPx, calcuatedGCPy = GCPxyFromExtrinsicParameters(GCPuv, GCPxyz[:,3], χ, ι)
    #
    ax.set_aspect("equal")
    ax.plot(χ.x_c, χ.y_c, "xr")
    ax.text(χ.x_c, χ.y_c, "camera", color="r")
    # Plot the provided xy coordinates:
    ax.plot(GCPxyz[:,1], GCPxyz[:,2], ".k", alpha=0.5)
    [ ax.text(GCPxyz[i,1], GCPxyz[i,2], i, alpha=0.5) for i in 1:size(GCPxyz,1) ]
    # Ploat the xy coordinates that were calculated from provided uv coordinates:
    ax.plot(calcuatedGCPx, calcuatedGCPy, "o", mfc="r", mec="r")
    ax.plot(calcuatedGCPx, calcuatedGCPy, ".k")
    [ ax.text(calcuatedGCPx[i], calcuatedGCPy[i], i, color="r", alpha=0.5) for i in 1:size(calcuatedGCPx,1) ]
    #
    for i in 1:size(GCPxyz,1)
        ax.plot([GCPxyz[i,1], calcuatedGCPx[i]], [GCPxyz[i,2], calcuatedGCPy[i]], "-k", lw=0.5, alpha=0.1)
        @info (GCPxyz[i,1], calcuatedGCPx[i])
    end
    #
    mean_eps = mean( hypot.( GCPxyz[:,1].-calcuatedGCPx, GCPxyz[:,2].-calcuatedGCPy ) )
    median_eps = median( hypot.( GCPxyz[:,1].-calcuatedGCPx, GCPxyz[:,2].-calcuatedGCPy ) )
    std_eps = std( hypot.( GCPxyz[:,1].-calcuatedGCPx, GCPxyz[:,2].-calcuatedGCPy ) )
    ax.set_xlabel("mean diff: $(round(mean_eps,digits=3))")
    return (μ=mean_eps, median=median_eps, σ=std_eps)
end
##############################################################################

##############################################################################
"""
Calculates GCP u,v (pixel) coordinates from provided GCP x,y,z (physical
coordinates), and provided intrinsic and extrinsic calibration parameters.

Plots calculated and provided GCP xy coordinates, and calculates the mean
distance between each GCP in the provided, and calculated, coordinate sets.
"""
function PlotGCPuvFromExtrinsicParameters(GCPuv, GCPxyz,
                                          χ::ExtrinsicCalibrationParameters,
                                          ι::IntrinsicCalibrationParameters;
                                          ax=plt.gca())
    calcuatedGCPu, calcuatedGCPv = GCPuvFromExtrinsicParameters(GCPxyz, χ, ι)
    #
    ax.set_aspect("equal")
    # Plot the provided uv coordinates:
    ax.plot(GCPuv[:,1], GCPuv[:,2], ".k", alpha=0.5)
    [ ax.text(GCPuv[i,1], GCPuv[i,2], i, alpha=0.5) for i in 1:size(GCPuv,1) ]
    # Ploat the uv coordinates that were calculated from provided xyz coordinates:
    ax.plot(calcuatedGCPu, calcuatedGCPv, "o", mfc="r", mec="r")
    ax.plot(calcuatedGCPu, calcuatedGCPv, ".k")
    [ ax.text(calcuatedGCPu[i], calcuatedGCPv[i], i, color="r", alpha=0.5) for i in 1:size(calcuatedGCPu,1) ]
    #
    for i in 1:size(GCPuv,1)
        ax.plot([GCPuv[i,1], calcuatedGCPu[i]], [GCPuv[i,2], calcuatedGCPv[i]], "-k", lw=0.5, alpha=0.1)
        @info (GCPuv[i,1], calcuatedGCPu[i])
    end
    ax.invert_yaxis()
    #
    mean_eps = mean( hypot.( GCPuv[:,1].-calcuatedGCPu, GCPuv[:,2].-calcuatedGCPv ) )
    median_eps = median( hypot.( GCPuv[:,1].-calcuatedGCPu, GCPuv[:,2].-calcuatedGCPv ) )
    std_eps = std( hypot.( GCPuv[:,1].-calcuatedGCPu, GCPuv[:,2].-calcuatedGCPv ) )
    ax.set_xlabel("mean diff: $(round(mean_eps,digits=3))")
    return (μ=mean_eps, median=median_eps, σ=std_eps)
end
###############################################################################
GCPuvFromExtrinsicParameters(GCPxyz, χ, ι) = xyz2uv(GCPxyz, DLT(χ, ι))
GCPxyFromExtrinsicParameters(GCPuv, GCPz, χ, ι) = uvz2xy(GCPuv, GCPz, DLT(χ, ι))
###############################################################################



GCPxyFromExtrinsicParameters (generic function with 1 method)

In [34]:
# SETH CAMERA CALIBRATION HELPER EXAMPLE
###############################################################################
# 
# CameraCalibrationHelpers.jl in /home/seth/.julia/dev/IRQIV/src/
# 
# Created: 2023-04-27 10:24:33 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-27 13:34:02 seth>
# 
###############################################################################

###############################################################################
# run this file to load packages and define the functions that we'll use:
###############################################################################
import Pkg
import PyPlot as plt
using Statistics
# push!(LOAD_PATH, raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\IRQIV.jl\examples")
# Pkg.activate(raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\IRQIV.jl\examples\CameraCalibrationHelpers_ETH.jl") # path to IRQIV directory
# using CameraCalibrationHelpers_ETH.jl
plt.pygui(true)
# include("CameraCalibrationHelpers.jl")


##############################################################################
# Run example using Sutter Slough calibration:
##############################################################################

# Define camera and its intrinsic parameters:
Camera = FLIR_SC8303_17mm
ι = IntrinsicCalibrationParameters(Camera)

# UTM coordinates of GCPs:
TrueGCPxyz = [
    624198.903   4243359.060   2.017
    624221.304   4243337.433   1.852
    624223.368   4243337.067   2.749
    624200.707   4243359.428   3.574
    624171.856   4243308.599   2.889
    624169.447   4243310.925   2.804
    624167.042   4243314.109   2.618
    624175.137   4243304.681   2.458]

# Pixel coordinates of GCPs, before accounting for radial distortion:
DistortedImageGCPuv  = [ 1295.0  113.4
                         1244.3  725.3
                         1265.6  757.45
                         1335.0  138.3
                         115.7  471.5
                         137.6  272.5
                         173.6   57.1
                         82.4  761.0]

## Remove radial distortion from GCP target coordinates:
# 
# (the hcat other things on this line are just to convert the output from a
# Tuple to an Array)
TrueGCPuv = hcat(RemoveRadialDistortion(DistortedImageGCPuv[:,1], DistortedImageGCPuv[:,2], Camera)...)

# Camera position initial guess:
Camera_x = 624165.196
Camera_y = 4243306.056
Camera_z = 24.24 - 0.4          # antenna was placed above camera, 0.4m offset
#
# estimated camera orientation angles:
φ =  50                         # azimuth
τ = 215                         # tilt
σ =  90                         # roll
#
f = -1200 # nominal_focal_length_px (nominal lens focal length / pixel pitch)

# Initial guess of camera's extrinsic parameters:
χ = ExtrinsicCalibrationParameters(
    deg2rad(φ),
    deg2rad(τ),
    deg2rad(σ),
    f,
    Camera_x,  # x_c
    Camera_y,  # y_c
    Camera_z) # z_c


##############################################################################
plt.close("all")
GraphicallyExploreInitialCalibrationGuess(TrueGCPuv, TrueGCPxyz, χ, ι)# ,
                                          # frame=fill(NaN, ι.v₀*2 - 1 |> Int, ι.u₀*2 - 1 |> Int))
##############################################################################

##############################################################################
# Optionally, load an IR image and plot it in the background:
FLIR_filename = joinpath(raw"D:\CNRD_IR",
                         "Rec-DeFreesLab_-003349.ats")
IRimages = LoadATSImageSequence(FLIR_filename, 1, 2017)
# 
plt.close("all")
GraphicallyExploreInitialCalibrationGuess(TrueGCPuv, TrueGCPxyz, χ, ι, IRimages[1])# ,
##############################################################################


[ Info: (624198.903, 624191.7649649581)
[ Info: (624221.304, 624203.1003668802)
[ Info: (624223.368, 624203.6389393692)
[ Info: (624200.707, 624192.4015051478)
[ Info: (624171.856, 624168.8956599968)
[ Info: (624169.447, 624167.0981831495)
[ Info: (624167.042, 624165.1666486443)
[ Info: (624175.137, 624171.5181389938)
[ Info: (1303.315387044818, 1499.1425577647958)
C:\Users\evanh\anaconda3\lib\site-packages\matplotlib\image.py:443: UserWarning: Warning: converting a masked element to nan.
  dv = np.float64(self.norm.vmax) - np.float64(self.norm.vmin)
C:\Users\evanh\anaconda3\lib\site-packages\matplotlib\image.py:444: UserWarning: Warning: converting a masked element to nan.
  vmid = np.float64(self.norm.vmin) + dv / 2
[ Info: (1250.1713727256965, 1524.8604917836303)
[ Info: (1274.865166910621, 1559.7197874384015)
[ Info: (1346.8148838272634, 1555.5924887039919)
[ Info: (116.63409264267179, 299.0113686652757)
[ Info: (139.22360912811874, 277.58757394470945)
[ Info: (172.3746028238008, 2

PyObject Text(0.5, 0.98, 'red is calculated, black is provided\n\nExtrinsicCalibrationParameters\n  φ: Float64 0.872665\n  τ: Float64 3.75246\n  σ: Float64 1.5708\n  f: Float64 -1200.0\n  x_c: Float64 6.24165e5\n  y_c: Float64 4.24331e6\n  z_c: Float64 23.84\n')

In [3]:
# SETH IRQIV EXAMPLE 
###############################################################################
# 
# IRQIV_MWE.jl in /home/seth/.julia/dev/IRQIV/test/
# 
# Created: 2023-04-24 16:55:53 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-26 15:08:17 seth>
# 
###############################################################################

###############################################################################
Pkg.activate(raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\IRQIV.jl\src\IRQIV") # path to IRQIV directory
using IRQIV
###############################################################################


###############################################################################
# 
### Select a dataset to operate on:
#
# Sutter Slough: ETH changed
FLIR_filename = joinpath(raw"D:\CNRD_IR",
                         "Rec-DeFreesLab_-003349.ats")
ats_year = 2022
@assert isfile(FLIR_filename)
###############################################################################

###############################################################################
const M=32
const N=32
const rhoI = 10
const rhoJ = 10
MQD_parameters = (M=M, N=N, rhoI=rhoI, rhoJ=rhoJ)
###############################################################################


###############################################################################
DtFrames = 4                    # Δt, frames (ATS 3116 is at 10Hz)
data_output_rate_frames = 10    # difference between I1 sequence, frames
nImagePairs = 100
###############################################################################


###############################################################################
# Load IR images:
I1inds = range(1, step=data_output_rate_frames, length=nImagePairs)
I2inds = I1inds .+ DtFrames
I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)
###############################################################################
# Remove image pairs when at least one image took place during NUC (calibration)
# mode:
if any(isnuc(I1images.Headers)) || any(isnuc(I2images.Headers))
    nuc_frames = isnuc(I1images.Headers) .| isnuc(I2images.Headers)
    I1inds = I1inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
    I2inds = I2inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
    I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
    I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)
end
###############################################################################

###############################################################################
subwincorners = CartesianIndices(( (N+rhoI):7:(size(I1images,1)-(N+rhoI+1)),
                                   (M+rhoJ):12:(size(I1images,2)-(M+rhoJ+1)) ))
###############################################################################


###############################################################################
# Initialize MQD calculation:
MQD_variables = initialize_MQD(I1images,
                               I2images,
                               subwincorners,
                               N, M,
                               rhoI, rhoJ)                              
###############################################################################

###############################################################################
# Perform the MQD analysis:
subpix_shifts_i, subpix_shifts_j, performance_message = perform_MQD(
    I1images, I2images, subwincorners, MQD_variables, MQD_parameters);
###############################################################################




  Activating new environment at `C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\IRQIV.jl\src\IRQIV\Project.toml`
1.0%┣▍                                         ┫ 1/100 [00:19<Inf:Inf, InfGs/it]
2.0%┣█                                             ┫ 2/100 [00:21<33:46, 21s/it]
3.0%┣█▍                                            ┫ 3/100 [00:22<17:39, 11s/it]
4.0%┣█▉                                             ┫ 4/100 [00:23<12:13, 8s/it]
5.0%┣██▍                                            ┫ 5/100 [00:24<09:28, 6s/it]
6.0%┣██▉                                            ┫ 6/100 [00:25<07:47, 5s/it]
7.0%┣███▎                                           ┫ 7/100 [00:26<06:41, 4s/it]
8.0%┣███▊                                           ┫ 8/100 [00:27<05:53, 4s/it]
9.0%┣████▎                                          ┫ 9/100 [00:28<05:15, 3s/it]
10.0%┣

LoadError: InterruptException:

████▌                                        ┫ 10/100 [00:29<04:47, 3s/it]
11.0%┣█████                                        ┫ 11/100 [00:30<04:27, 3s/it]
12.0%┣█████▍                                       ┫ 12/100 [00:31<04:07, 3s/it]
13.0%┣█████▉                                       ┫ 13/100 [00:32<03:51, 3s/it]
14.0%┣██████▎                                      ┫ 14/100 [00:33<03:37, 3s/it]
15.0%┣██████▊                                      ┫ 15/100 [00:34<03:25, 2s/it]
16.0%┣███████▏                                     ┫ 16/100 [00:35<03:14, 2s/it]
17.0%┣███████▋                                     ┫ 17/100 [00:36<03:04, 2s/it]
18.0%┣████████                                     ┫ 18/100 [00:36<02:56, 2s/it]
19.0%┣████████▌                                    ┫ 19/100 [00:37<02:48, 2s/it]
20.0%┣█████████                                    ┫ 20/100 [00:38<02:41, 2s/it]
21.0%┣█████████▌                                   ┫ 21/100 [00:39<02:35, 2s/it]
22.0%┣██████████                  